# Menadione Tn-seq notebook

In [ ]:
#download from genewiz sftp
sftp jstien01_student_bbk@gweusftp.azenta.com
lcd /d/in16/u/sj003/men_tnseq
cd 40-842749567
mget *

In [21]:
# files in /d/in16/u/sj003/men_tnseq/fastq

#!ls fastq
zcat A1_R1_001.fastq.gz | head -5
zcat A1_R2_001.fastq.gz

gzip: A1_R1_001.fastq.gz: No such file or directory
gzip: A1_R2_001.fastq.gz: No such file or directory


### File checks to make sure downloaded correctly--matching sizes?

In [ ]:
#check length of files to see if reads same in all files?
!cd fastq
!FILES=*.fastq.gz
!for file in $FILES; do wc -l $file; done >> sanity_check.txt
# all 3 files have different line counts, R2 is index. R1 and R3 more similar, different because of line continuation?

# check files downloaded correctly
!for file in $FILES; do md5sum -c $file.md5; done >> md5_check.txt

#all 'OK'

#head
!for file in $FILES; do echo $file; zcat $file | head -10 $file; done


line count was the same for R1 and R2 when we did cattle in vivo

In [8]:
# for A1 sample see how many times header appears in file
!cd fastq
!zgrep -c 'N:0:AACGTGAT' A1_R1_001.fastq.gz
#64796903
!zgrep -c 'N:0:AACGTGAT' A1_R2_001.fastq.gz
#64796903
!zgrep -c 'N:0:AACGTGAT' A1_R3_001.fastq.gz
#64796903

gzip: A1_R1_001.fastq.gz: No such file or directory


ON mac the zgrep command doesn't seem to work (or zcat?) use gunzip?

This indicates same number of reads in all files.

Run fastqc and then can double-check read length for all files

In [20]:
!cd fastq
!module load fastqc
!module load multiqc
!FILES=*.fastq.gz
!for f in $FILES; do fastqc ${f} -o fastqc; done
!cd fastqc
!multiqc .


SyntaxError: invalid syntax (<ipython-input-20-a2cea0d7dfa0>, line 2)

In [1]:
#test bash command

pwd
FILES=*fastq.gz
for file in $FILES; do echo $file; done

SyntaxError: invalid syntax (<ipython-input-1-03ca3f647ae6>, line 5)

These look fine--reads of 151bp for R1 and R3 (corresponding to Read 1 and Read 3) and i7 read (R2) is 8bp which represents the barcode.

Next step is to iterate through reads in R1 and R2 to 'demultiplex' by combining to one 'template' the reads that have the same barcode in R2.


In [6]:
!zcat fastq/A1_R2_001.fastq.gz | head -20

@A01968:63:H77VYDSX5:4:1101:28212:1031 2:N:0:AACGTGAT
CGCAGTAT
+
FFFFFF:F
@A01968:63:H77VYDSX5:4:1101:29369:1031 2:N:0:AACGTGAT
GAGCACCT
+
FF:FFFFF
@A01968:63:H77VYDSX5:4:1101:32027:1031 2:N:0:AACGTGAT
CATACCAA
+
FFFFFFFF
@A01968:63:H77VYDSX5:4:1101:3341:1047 2:N:0:AACGTGAT
GTACATGC
+
:FFFF::F
@A01968:63:H77VYDSX5:4:1101:9073:1047 2:N:0:AACGTGAT
GCCGCCCC
+
FFFFFFFF

gzip: stdout: Broken pipe


Line 1 is the read identifier with the barcode

In [51]:
!zcat fastq/A1_R1_001.fastq.gz | head -20
!zcat fastq/A1_R1_001.fastq.gz | head -1000 > test_1000_R1.txt

zcat: can't stat: fastq/A1_R1_001.fastq.gz (fastq/A1_R1_001.fastq.gz.Z): No such file or directory


Testing on laptop need to install gnused before can use sed

In [41]:
# find all unique reads in R2 line 2 of each read
#!head -1000 data/fastq/A1_R2_001.fastq > test_1000_R2.txt
#!brew install gnu-sed
#!sed -n '1~2p' test_1000_R2.txt | sort | uniq -c | sort -nr
# need gsed installed for this
#!awk 'NR % 2 == 0' data/fastq/test_1000_R2.txt | sort | uniq -c | sort -nr
# print every 4 lines starting from line 2
!awk 'NR % 4 == 2' data/fastq/test_1000_R2.txt > test.txt
cat test.txt


In [44]:
!sort test.txt | uniq -c | sort -nr

  12 GGGGGGGG
   3 ACAAGAAT
   2 GCCGCCCT
   2 CGGCGTCA
   2 ATTGAAGT
   2 ATATACAA
   2 ACTATCGT
   2 AATTAACT
   2 AAGGTTAA
   1 TTTTGAGG
   1 TTTTACCT
   1 TTTAGTCA
   1 TTGCACGA
   1 TTCGTGCA
   1 TTCGTACT
   1 TTCCCAGT
   1 TTCATAAG
   1 TTCACGTC
   1 TTAGTAAC
   1 TTAGACAT
   1 TTAAAACC
   1 TGTTGTGA
   1 TGTTGTCT
   1 TGTCCATC
   1 TGGTTCAA
   1 TGCACTTT
   1 TGCAACGG
   1 TGATCTTA
   1 TGAGCACA
   1 TGACATGA
   1 TGACATCT
   1 TGACATAT
   1 TCTTGTCA
   1 TCTGCCTC
   1 TCTACTTC
   1 TCTAACGC
   1 TCGTTCAG
   1 TCGTCTTA
   1 TCGTCCAG
   1 TCATTGAA
   1 TCATTCAT
   1 TCATGTTA
   1 TCACATGC
   1 TCACAAGT
   1 TCAATTAC
   1 TCAACAGC
   1 TATTAATT
   1 TATTAAGC
   1 TATGATCT
   1 TAGTCAAG
   1 TAGTATTT
   1 TAGCTGAT
   1 TAGCATAA
   1 TAGACTCA
   1 TACTTGAC
   1 TACTATTC
   1 TACGATAC
   1 TAATCATC
   1 TAAGACCC
   1 TAAAATCA
   1 GTTCTGCT
   1 GTGCCGAG
   1 GTGACCCA
   1 GTGACACG
   1 GTCTAATT
   1 GTCCACAT
   1 GTCCAACT
   1 GTAGCATG
   1 GTACCGAT
   1 GTACATGC
   1 GTAATGCA
   1 G

This shows how many duplicates of certain barcodes. need to match barcodes with read name to eliminate ones that have repeats

In [47]:
#do again with whole file (takes a really long time)
!awk 'NR % 4 == 2' data/fastq/A1_R2_001.fastq | sort | uniq -c | sort -nr > count_barcodes.txt

# top counts
1312199 GGGGGGGG
16805 CAAAACGA
16371 GCAACATT
14835 AATACAAC
14507 CCAAAGAC
14483 AATATAAC
14427 TAACAACA
14120 AAAACATA
13989 TAAAACTA
13544 CCGAAACA

Find distribution of these reads

Which reads are associated with these really big duplications?

In [50]:
# thiss just puts pattern in file--not whole read
!grep 'GGGGGGGG' data/fastq/A1_R1_001.fastq > gggg_dupl.fastq

paste two files together so barcode is added onto read header?

In [ ]:
#bash solution
!paste -d " " file1.txt file2.txt

Make a new fastq file 

In [127]:
#python
seq_list = []
with open ('test_1000_R1.txt', 'r') as f1, open('test_1000_R2.txt', 'r') as f2:
    #line1_f1 = f1.readline()
    #for line in f1.read().split("\n")[0]:
    #    print(line.rstrip())
    # header and sequence for R1 reads
    for lineno, line in enumerate(f1):  
        if lineno % 4 == 0:
            f1_head = line.rstrip()
        # second line and every 4  (sequence)  
        if lineno %4 == 1:
            #print(line)
            f1_seq = line
        # spacer line
        if lineno %4 ==2:
            f1_spacer = line
        #quality line
        if lineno %4 ==3:
            f1_quality = line

        
    #create header and barcode for index reads
    for lineno, line in enumerate(f2):
        if lineno % 4 == 0:
            f2_head = line.rstrip()
            
        if lineno % 4 == 1:
            #print(line)
            f2_barcode = line.rstrip()
        # add barcode to end of header in read1 file
        new_head = f1_head + " BC:" + f2_barcode

        list_entry = new_head + "\n" + f1_seq + f1_spacer + f1_quality
    
        seq_list.append(list_entry)

# write new file with barcodes
with open('new_barcode_file_1000.txt', 'w') as outfile:
    outfile.writelines(seq_list)
outfile.close()

Looking at the sequences for the R1 reads that have the GGGGGGGG 'random barcode' in the first 1000 reads, these sequences all have huge runs of G's?

"For Illumina NextSeq/NovaSeq data, polyG can happen in read tails since G means no signal in the Illumina two-color systems. fastp can detect the polyG in read tails and trim them. This feature is enabled for NextSeq/NovaSeq data by default, and you can specify -g or --trim_poly_g to enable it for any data, or specify -G or --disable_trim_poly_g to disable it. NextSeq/NovaSeq data is detected by the machine ID in the FASTQ records. " 
Need to map reads first because will exclude artefactual PCR amplicons that have identical P7 as well as same genomic insertion site. (from fastp documentation, https://github.com/OpenGene/fastp)

But maybe need to add the barcode to the header of each read first. Adapt above to make python script that can work on entire files of reads and make new files

In [1]:

def add_barcode(file1, file2):
    """
    A function to extract the barcode from the associated p7 index read to the header of each read in the fastq file

    Input               file1                       fastq file for sequence reads (R1 or R3)
                        file2                       fastq file for i7 index reads (R2)
    Output              barcoded<sample>.fastq      new fastq file which has barcode added to header   

    """
    import re
    import os.path
    import sys

    # identify sample name
    filename = str(file1)
    bn_sample = os.path.basename(file1)
    sample = re.sub(".fastq", "", bn_sample)
    print(sample)

    seq_list = []
    with open (file1, 'r') as f1, open(file2, 'r') as f2:
        #count lines in R1 and R2 and proceed if equal
       # if sum(1 for _ in f1) != sum(1 for _ in f2):
        #    sys.exit("Number of reads do not match")
            
    # header and sequence for R1 reads
        for index, (line1, line2) in enumerate(zip(f1, f2)):
            if index % 4 == 0:
                f1_head = line1.rstrip()
                f2_head = line2.rstrip()
            # second line and every 4  (sequence)  
            if index %4 == 1:
                f1_seq = line1.rstrip()
                #create header and barcode for index reads
                #>A01968:63:H77VYDSX5:4:1101:25455:1423 1:N:0:AACGTGAT BC:GGGGGGGG
                f2_barcode = line2.rstrip()
                # add barcode to end of header in read1 file
                new_head = f1_head + "_BC:" + f2_barcode
                #replace whitespace
                new_head = new_head.replace(" ", "_")
                
            # don't need these if do after trimming
            #quality line
            #if index %4 ==3:
                #f1_quality = line1.rstrip()
                #list_entry = new_head + "\n" + f1_seq + "\n" + "+" + "\n" + f1_quality + "\n"
                list_entry = new_head + "\n" + f1_seq + "\n"
                seq_list.append(list_entry)
                seq_list.append(list_entry)
    print(len(seq_list))
    # write new file with barcodes
    new_filename = "barcode_" + sample + ".fastq"
    with open(new_filename, 'w') as outfile:
        outfile.writelines(seq_list)
    outfile.close()


In [2]:
add_barcode("test_5000_R1.fastq", "test_5000_R2.fastq")

test_5000_R1
1250


In [97]:
add_barcode("data/fastq/A1_R1_001.fastq", "data/fastq/A1_R2_001.fastq")

A1_R1_001


In [183]:
# can also apply to read2, but no point as won't be using for mapping
#add_barcode("data/fastq/A1_R3_001.fastq", "data/fastq/A1_R2_001.fastq")

A1_R3_001
64796903
barcode_A1_R3_001.fastq


Map small file to see if header stays connected to read (installed new bwa-mem2)


In [ ]:
# sample first 10000 bases of barcoded fastq
!head -10000 barcode_A1_R1_001.fastq > new_barcode_file_A1_10000.fastq 


In [ ]:
!conda activate tnseq
# have to index file first for bwa-mem2
!bwa-mem2 index ref_seqs/Mbovis_AF2122-97.fasta
!bwa-mem2 mem -C ref_seqs/Mbovis_AF2122-97.fasta barcode_test_5000_R1.fastq > test_map_5000.sam
# still getting error: ERROR! Unable to open the file: ref_seqs/Mbovis_AF212297.fasta.bwt.2bit.64
# there were multiple fasta files, deleted all others and manually coppied and pasted text from ref file. changed to Mbovis_AF2122-97.fasta




barcode doesn't appear in sam header automatically, -C to append FASTA/FASTQ comment to SAM output (appends to end of file--column 12 in sam file) (-V            output the reference FASTA header in the XR tag--gives blank file)

Column 4 in sam header is meant to be the start mapping position (should be insert position). in test file, all have start position as 0--probably didn't map?

In [13]:
#parse sam file for start and barcode

def parse_samfile(samfile):
    import pandas as pd
    """
    find molecular barcode and start of mapping in each read
    """

    sam_list = []
    barcodes = []
    id_dups = []
    data = pd.read_csv(samfile, sep="\t", skiprows=2, header=None)
    #print(data.head())
    ids = data[0]
    starts = data[3]
    # sometimes dividing up final lines? 13 for 1000 or 15 for 100000
    barcodeLines = list(data[15])
    #extract molecular barcode
    for line in barcodeLines:
        barcodes.append(line.split("BC:",1)[1])
    # remove non-unique rows?
    #dup_tup = pd.DataFrame(ids)
    dup_tup = pd.DataFrame(starts)
    dup_tup.columns=["starts"]
    dup_tup['barcodes'] = barcodes
    # get count of duplicates for each unique row
    sorted_dup_tup = dup_tup.groupby(dup_tup.columns.tolist(), as_index=False).size()
    
    return sorted_dup_tup
    
    
p = parse_samfile("test_map_10000.sam")
print(p.head())

    starts  barcodes  size
0  2736492  AAAAAAAT     1
1  2736492  AAAAAAGG     1
2  2736492  AAAAACAG     1
3  2736492  AAAAACAT     1
4  2736492  AAAAACCT     1


column 3 (0 indexed) is start, column 13/15 has both barcodes, column 9 is sequence. No duplicate start/barcode combos apparent in the 10000bp test file.

Better approach may be to process reads and find duplicates using Picard find duplicates tool (Mark Duplicates)

https://gatk.broadinstitute.org/hc/en-us/articles/360037051452-EstimateLibraryComplexity-Picard-

can use argument --BARCODE_TAG 'BC' to identify barcode
--READ_NAME_REGEX null if duplicate sets are very high and we are not trying to establish library complexity

Reads must be mapped and sorted

In [ ]:
!conda install -c bioconda picard
!java -jar picard.jar MarkDuplicates \
      I=input.bam \
      O=marked_duplicates.bam \
      M=marked_dup_metrics.txt

First step before mapping is to remove the transposon sequence from each read
## From TRANSIT/TPP docs:

1. Convert .fastq files to .fasta format (.reads).
    https://bioinformaticsworkbook.org/dataWrangling/fastaq-manipulations/converting-fastq-format-to-fasta.html#gsc.tab=0
    This would only keep header and sequence.

(creates .reads file using 'fastq2reads')

In [176]:
gsed -n '1~4s/^@/>/p;2~4p' new_barcode_file_1000.fastq > new_barcode_file_1000.fasta


2. Identify reads with the transposon prefix in R1 . The sequence searched for is ACTTATCAGCCAACCTGTTA (or TAAGAGACAG for Tn5), which must start between cycles 5 and 10 (inclusive). (Note that this ends in the canonical terminus of the Himar1 transposon, TGTTA.) The “staggered” position of this sequence is due to insertion a few nucleotides of variable length in the primers used in the Tn-Seq sample prep protocol (e.g. 4 variants of Sol_AP1_57, etc.). The number of msmatchen searching reads for the transposon sequence pattern can be adjusted as an option in the interface; the default is 1.

    Use fastp for this? First try with TPP?


In [ ]:
#(thoth)
#!tpp -bwa /s/software/bwa/bwa/bwa -ref $my_path/refseqs/Mtb/Mtb_H37Rv.fasta \
#    -reads1 $my_path/ncbi/files/dejesus/SRR4113428_1.fastq \
#        -reads2 $my_path/ncbi/files/dejesus/SRR4113428_2.fastq \
#            -output $my_path/dejesus_mtb/tpp_results/SRR4113428

# paired-end (laptop)
# this will look for barcode which isnt' there, but maybe useful to trim read 2 and extract genomic sequence?
!tpp -bwa ~/anaconda3/envs/tnseq/bin/bwa-mem2 -ref ref_seqs/Mbovis_AF2122-97.fasta \
    -reads1 barcode_A1_R1_001.fastq -reads2 data/fastq/A1_R3_001.fastq \
        -output tpp_test_A1_paired

# Error: unexpected format of headers in .fastq files
#Assume this is due to adding barcode to header for read1



Fastp can use index reads for UMI processing--can extract from index reads and append to first part of read names so it will be in sam/bam files, can specify prefix 'UMI'

In [181]:
!fastp -i data/fastq/A1_R1_001.fastq -I data/fastq/A1_R3_001.fastq -o data/fastp_out/A1_R1_001_trimmed.fastq -O data/fastp_out/A1_R3_001_trimmed.fastq \
 -U --umi_loc=index1 --umi_prefix=UMI --unpaired1 -l=20 --adapter_fasta adapter_file.txt

#It reads from index that is already in multiplexed header--not from index file, so not useful unless specify P7 reads as R2 and it can get umi from read2
#--unpaired1 means will save unpaired read 1s, -l 20 means will require reads to be at least 20 bp after trimming

zsh:1: command not found: fastp


Added an 'adapter file' to use with fastp: 'adapter_file.txt' which includes the transposon sequence pattern for R1 and R2.

This is leading to very short R1 sequences which actually match the transposon sequence (no genomic?) Are R1 and R3 reversed and R3 is really read1?

Do tests with shorter number of reads (100000)

Use R2 index reads as read2?

#temporarily disable adapter trimming 

In [ ]:
#fastp -i data/fastq/A1_R1_001.fastq -I data/fastq/A1_R2_001.fastq -o data/fastp_out/A1_R1_001_trimmed.fastq -O data/fastp_out/A1_R3_001_trimmed.fastq \
 #-U --umi_loc=read2 --umi_prefix=UMI --umi_len=8 --unpaired1 -l 20 --adapter_fasta adapter_file.txt --reads_to_process=100000

fastp -i data/fastq/A1_R1_001.fastq -I data/fastp_out/A1_R2_001_trimmed.fastq \ 
-U --umi_loc=read2 --umi_prefix=UMI --umi_len=8 --reads_to_process=100000 -o data/fastp_out/A1_R1_001_trimmed.fastq -O data/fastp_out/A1_R3_001_trimmed.fastq
 
 #this doesn't work--needs to have properly paired ends

This is new heading that fastp makes that includes UMI: @A01968:63:H77VYDSX5:4:1101:32027:1031:UMI_AACGTGAT 1:N:0:AACGTGAT
I imagine this will work for TPP and still have heading included in mapped reads if I just move it to end of first field instead of randomly adding to end. MIght be easier to filter with picard then. but not sure it matters if I don't do tpp since I can parse from sam files myself--it is only using P5 index from the header of the reads--NOT UMI barcode from p7 index files

In [ ]:
#unpaired fastp for adapter trimming
!fastp -i new_barcode_file_A1_10000.fastq -o data/fastp_out/test_A1_R1.fastq --adapter_fasta adapter_file.txt

This is just leaving super short reads like before (15 bp long--also still part of primer, no genomic sequence left). Maybe because I have both ends of inverted repeat sequence in there

@A01641:207:HNJLJDSX3:1:2678:31331:37059 1:N:0:AACGTGAT BC:CGCAGTAT
GTCTAGAGACCGGGG
+
F::FF:FFFFFFFFF

In [ ]:
# without adapter fasta file--using transposon seq for R1 only
!fastp -i data/fastq/A1_R1_001.fastq -o data/fastp_out/test_repeat_A1_R1.fastq --adapter_sequence=ACTTATCAGCCAACCTGTTA --reads_to_process=100000
#results in mostly 15 bp identical sequences
#reads with adapter trimmed: 98991 (trims nearly all)

#try with R3
!fastp -i data/fastq/A1_R3_001.fastq -o data/fastp_out/test_repeat_A1_R3.fastq --adapter_sequence=ACTTATCAGCCAACCTGTTA --reads_to_process=100000
#reads with adapter trimmed: 17

#using Read 2 transposon sequence:
!fastp -i data/fastq/A1_R3_001.fastq -o data/fastp_out/test_repeat_A1_R3.fastq --adapter_sequence=TGGTCGTGGTAT --reads_to_process=100000
#bases trimmed due to adapters: 7559

# using rc of read 2 transposon sequence:
!fastp -i data/fastq/A1_R3_001.fastq -o data/fastp_out/test_repeat_A1_R3.fastq --adapter_sequence=TACCACGACCA --reads_to_process=100000


# using automatic detection
!fastp -i data/fastq/A1_R1_001.fastq -o data/fastp_out/test_automatic_A1_R1.fastq --reads_to_process=100000
#finds truseq adapter read1, but not many are actually trimmed

First trim with sequencing adapters. Then do again in separate step (cutadapt) to cut off transposon prefix

added barcode info to both paired-end reads (R1 and R3). After mapping will try to remove duplicates and make wigs again? Will have to put into header of both reads as TPP needs these to match.

Paired end takes hours for single sample. Not going to be useful as R2 contains empty barcodes. Use single-end to see if get anything useful. otherwise do manually and compare with trimmed files 

In [ ]:
#tpp on paired ends with matched headers. doesn't map with no barcodes on 2nd read (set for paired end#)
!tpp -bwa ~/anaconda3/envs/tnseq/bin/bwa-mem2 -ref ref_seqs/Mbovis_AF2122-97.fasta \
    -reads1 barcode_A1_R1_001.fastq -reads2 barcode_A1_R3_001.fastq -output tpp_test_A1_paired

# try for single end 
#!tpp -bwa ~/anaconda3/envs/tnseq/bin/bwa-mem2 -ref ref_seqs/Mbovis_AF2122-97.fasta \
#    -reads1 barcode_A1_R1_001.fastq -output tpp_test_A1_single

#on thoth
!tpp -bwa /s/software/bwa/bwa/bwa -ref /d/in16/u/sj003/refseqs/mbovis/Mbovis_AF2122_97.fasta \
    -reads1 fastq/A1_R1_001.fastq.gz -output tpp/tpp_test_A1_single
#TypeError: sequence item 1: expected str instance, bytes found
#gunzipped file and tried again with unzipped file--seemed to work, but want to use barcoded fastq instead so aborted
#!gunzip fastq/A1_R1_001.fastq.gz
#!tpp -bwa /s/software/bwa/bwa/bwa -ref /d/in16/u/sj003/refseqs/mbovis/Mbovis_AF2122_97.fasta \
#    -reads1 fastq/A1_R1_001.fastq -output tpp/tpp_test_A1_single

#zip file on laptop, send to thoth, try again
!gzip barcode_A1_R1_001.fastq 
!scp barcode_A1_R1_001.fastq.gz sj003@ssh.cryst.bbk.ac.uk:/d/in16/u/sj003/men_tnseq/fastq/
!gunzip fastq/barcode_A1_R1_001.fastq.gz
!tpp -bwa /s/software/bwa/bwa/bwa -ref /d/in16/u/sj003/refseqs/mbovis/Mbovis_AF2122_97.fasta \
    -reads1 fastq/barcode_A1_R1_001.fastq -output tpp/tpp_test_A1_single

#barcoded reads were wrong--all had same sequence information (bug in program)




3. Extract genomic part of read 1. This is the suffix following the transposon sequence pattern above. However, for reads coming from fragments shorter than the read length, the adapter might appear at the other end of R1, TACCACGACCA. If so, the adapter suffix is stripped off. (These are referred to as “truncated” reads, but they can still be mapped into the genome just fine by BWA.) The length of the genomic part must be at least 20 bp.

4. Extract barcodes from read 2. Read 2 is searched for GATGGCCGGTGGATTTGTGnnnnnnnnnnTGGTCGTGGTAT”. The length of the barcode is typically 10 bp, but can be varaible, and must be between 5-15 bp.
    -can tpp skip this step (it takes absolutely forever)?
    -can this be done manually and fed in? or can I use header from .sam files in step 8 instead of barcode

5. Extract genomic portions of read 2. This is the part following TGGTCGTGGTAT…. It is often the whole suffix of the read. However, if the read comes from a short DNA fragment that is shorter than the read length, the adapter on the other end might appear, in which case it is stripped off and the nucleotides in the middle representing the genomic insert, TGGTCGTGGTATxxxxxxxTAACAGGTTGGCTGATAAG. The insert must be at least 20 bp long (inserts shorter than this are discarded, as they might map to spurious locations in the genome).
    -can this work in paired end without barcodes? 

6. Map genomic parts of R1 and R2 into the genome using BWA. Mismatches are allowed, but indels are ignored. No trimming is performed. BWA is run in ‘sampe’ mode (treating reads as pairs). Both reads of a pair must map (on opposite strands) to be counted.
    

7. Count the reads mapping to each TA site in the reference genome (or all sites for Tn5).

8. Reduce raw read counts to unique template counts. Group reads by barcode AND mapping location of read 2 (aka fragment “endpoints”).
    -i can do this with mapped reads


Read 1 is successfully trimmed using himar1 sequence. Read2 isn't trimmed (can't find himar1 sequence). Can use these files for ds analysis? (_paired.trimmed1). This is final fastq for read1 for tpp. _paired.genomic2 is final file for read 2.

I should maybe map using .trimmed1 and .trimmed2? or run tpp in single-end mode only

Mendum, smith, butler and that group only gets single-end reads from sequencing. OK to ignore read 2

Alternate (non-tpp) strategy
1. add barcodes to read 1 from index reads 
2. fastp for illumina trimming and QC (.fastq, filtered and trimmed)
3. then map with bwa-mem2 (.sam/.bam)
4. then filter reads for ...TGGTA in first 32 bp of read and start/barcode (filtered and mapped)
5. then count template reads mapping to each TA site (generate .wig files)
6. use transit for essentiality/resampling

## pipeline

1. add barcodes to Read1 from index reads


In [ ]:
add_barcode(READ1, INDEX)


2. fastp for qc and illumina adapter trimming

I believe the reads have already been trimmed by genewiz when multiplexed as fastp doesnt detect many of the sequencing adapters. Could skip trimming, but useful for shortening test file and also qc and catching any untrimmed adapters.


In [ ]:
#automatic adapter trimming
#!fastp -i barcode_A1_R1_001.fastq.gz -o trimmed/test_A1_R1_trimmed.fastq --reads_to_process=10000
# this trims all reads and nothing left!
#Detecting adapter sequence for read1...
#GTCTAGAGACCGGGGACTTATCAGCCAACCTGTTAACCGAGGCATCCCAGAGCTTGCCGG

#this is trasposon sequence and also genomic dna. skip trimming and map without? more aggressive trimming using barcode processed file

#disable adapter trimming or indicate only truseq adapter

!fastp -a GATCGGAAGAGCACACGTCTGAACTCCAGTCAC -i barcode_A1_R1_001.fastq -o trimmed/test_A1_R1_trimmed.fastq --reads_to_process=10000

#on thoth
!fastp -a GATCGGAAGAGCACACGTCTGAACTCCAGTCAC -i fastq/barcode_A1_R1_001.fastq -o trimmed/barcode_A1_R1_001.fastq

3. Map read 1 with bwa-mem2 and sort
    -can use TPP single end for this?

    Use snakemake mapping/sorting script

In [ ]:
# !conda activate tnseq
# # have to index file first for bwa-mem2
# !bwa-mem2 index ref_seqs/Mbovis_AF2122-97.fasta
# !bwa-mem2 -t 3 -C ref_seqs/Mbovis_AF2122-97.fasta data/fastp_out/A1_R1_trimmed.fastq > mapped_reads/A1_R1.sam
# # bwa samse?
# #!samtools view -Sb mapped_reads/A1_R1.bam
# !samtools sort -T -o sorted_reads/A1_R1_sorted.sam -O sam mapped_reads/A1_R1.sam 


# with snakemake (maps, sorts, indexes and creates flagstats report)
#make config.yaml file


!cd ~/tn_seq/menadione_tnseq/
!conda activate snakemake
!snakemake -np -s ~/snakemake/tnseq/snakefile.smk
!snakemake --cores 2 -s ~/snakemake/tnseq/snakefile.smk
#!snakemake -np -s $my_path/snakemake/tnseq/snakefile.smk
#!nohup snakemake --cores 8 -s $my_path/snakemake/map_bwa/pe/snakefile.smk > nohup_map.out 2>&1 &

samtools doesn't recognise the .sam format made by bwa in the -C mode includes quality scores, etc? When transit adds barcodes, it does it to names when making read files, so I've moved barcode to name so I don't have to worry about keeping any fields after the sequence.

More info about -C parameter and formation of sam headers:
"Append append FASTA/Q comment to SAM output. This option can be used to transfer read meta information (e.g. barcode) to the SAM output. Note that the FASTA/Q comment (the string after a space in the header line) must conform the SAM spec (e.g. BC:Z:CGTAC). Malformated comments lead to incorrect SAM output."

except for barcode, all my reads are identical. maybe this is something wrong with the barcoded file--since fastp also removed whole read.

Yes--barcode script was wrong. fixed bug. This works for downstream snakemake trimming/mapping pipeline

Filter out unmapped reads--part of snakemake pipeline. Changed snakemake to include header (@SQ) in output .sam file

4. filter reads for transposon sequence (in first 32 bp?)
 

 

In [17]:
#slice <first 32 bp of sequence line> | grep <transposon tag> > concatenate to .tag_positive file



#filter reads in samfile for transposon tag in sequence
#from tpp (https://github.com/mad-lab/transit/blob/master/src/pytpp/tpp_tools.py)
# find index of H[1..m] in G[1..n] with up to max mismatches
# note: this find first match, not necessarily the best (with min mismatches)

def mmfind1(G,n,H,m,max): # lengths; assume n>m
  """
  A function to find matches of transposon sequence in read
    Input               G                           sequence string
                        n                           length of read sequence
                        H                           pattern string
                        m                           length of pattern
                        max                         maximum mismatches
    Output              i, -1                       start position of match, or -1 for no match
  """

  a = G[:n].find(H[:m])
  if a!=-1: return a # shortcut for perfect matches
  for i in range(0,n-m):  # range of 0 to difference between seq length and len pattern
    cnt = 0
    for k in range(m):
      if G[i+k]!=H[k]: cnt += 1
      if cnt>max: break
    if cnt<=max: return i
  return -1
    
def find_tags(samfile, target_tag, mismatch_max=2):
    
    """
    Find transposon tag in sequence of each read.

      Input           samfile           file of mapped reads in .sam format
                      target_tag        string that matches transposon sequence
                      mismatch_max      number of mismatches allowed
      Output          match_list        list of length of number of reads in .sam file 
                                        with match start position or -1 if no match
    """

    import pandas as pd
    import pickle
    import os

    match_list = []
    # this is giving errors in parsing
    data = pd.read_csv(samfile, sep="\t", comment="@", header=None, usecols=[0,1,2,3,4,5,6,7,8,9,10])
    seqs = data[9]
    #search string for transposon seq with 2 mismatches
    for i in range(0, len(seqs)):
      seq = seqs[i]
      res = mmfind1(seq, len(seq), target_tag, len(target_tag), mismatch_max)
      match_list.append(res)
    #print(len(match_list))
    print("Number of reads with no transposon tag: ", match_list.count(-1))
    print("Number of reads with tag within first 32 bases: ", sum(1 for i in match_list if i >0 and i < 32))
    bn = os.path.basename(samfile)
    file_name = bn + "_tags" + ".pkl"
    with open(file_name, 'wb') as f:
      pickle.dump(match_list, f)
    return match_list
      


In [32]:
find_tags("sorted_reads/test_5000_R1.sam", "ACTTATCAGCCAACCTGTTA", 2)

Number of reads with no transposon tag:  611
Number of reads with tag within first 32 bases:  635


[-1,
 15,
 15,
 -1,
 15,
 -1,
 15,
 15,
 -1,
 15,
 -1,
 14,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 15,
 -1,
 15,
 15,
 -1,
 15,
 15,
 15,
 -1,
 15,
 -1,
 -1,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 -1,
 15,
 15,
 -1,
 15,
 15,
 15,
 15,
 15,
 -1,
 -1,
 15,
 -1,
 -1,
 -1,
 15,
 15,
 -1,
 -1,
 15,
 15,
 15,
 15,
 -1,
 15,
 15,
 -1,
 15,
 -1,
 -1,
 15,
 15,
 -1,
 -1,
 15,
 -1,
 -1,
 15,
 15,
 -1,
 -1,
 -1,
 -1,
 15,
 -1,
 -1,
 15,
 -1,
 15,
 15,
 15,
 15,
 -1,
 -1,
 15,
 15,
 -1,
 15,
 15,
 15,
 14,
 15,
 -1,
 15,
 15,
 -1,
 -1,
 -1,
 -1,
 15,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 15,
 15,
 15,
 15,
 -1,
 -1,
 -1,
 15,
 15,
 15,
 15,
 15,
 -1,
 -1,
 15,
 15,
 -1,
 15,
 -1,
 -1,
 15,
 -1,
 -1,
 15,
 15,
 -1,
 -1,
 -1,
 15,
 -1,
 -1,
 -1,
 -1,
 15,
 -1,
 -1,
 15,
 15,
 15,
 -1,
 15,
 15,
 -1,
 15,
 -1,
 15,
 -1,
 -1,
 -1,
 15,
 15,
 15,
 15,
 -1,
 15,
 15,
 -1,
 15,
 15,
 15,
 -1,
 15,
 -1,
 -1,
 -1,
 -1,
 15,
 -1,
 15,
 -1,
 15,
 15,
 15,
 15,
 15,
 -1,
 15,
 -1,
 -1,
 -1,
 -1,
 15,
 -1,


In [4]:
def find_barcodes(read):
    """
    find molecular barcode and start of mapping in each read
    """
    #extract molecular barcode from a single read and return tuple
    
    read_name = read.split()[0]
    barcode = read_name.split("BC:",1)[1]
    read_start = read.split()[3]
    bar_start = (barcode, read_start) 
    return bar_start



In [5]:



def filter_mapped_reads(sam_file, tag="ACTTATCAGCCAACCTGTTA", mis_max=2, position_pkl=[]):
  """ 
  """
  import sys
  import pickle
  import os
  import pandas as pd
  from operator import itemgetter
  if position_pkl == []:
    #create pos_list
    pos_list = find_tags(sam_file, tag, mis_max)
  else:
    with open(position_pkl, 'rb') as f:
      pos_list = pickle.load(f)
  #read sam_file and sort lines between header and reads
  header = []
  reads  = []
  barcode_list = []
  good_reads = []
  notags_reads = []
  bad_reads = []
  with open(sam_file, 'r') as f:
    for line in f:
      line = line.strip()
      if line[0] == "@":
        header.append(line)
      else:
        reads.append(line)
  #compare number of reads in sam file to positions in pos_list
  if len(reads)==len(pos_list):
    for i in range(0, len(pos_list)):
      if pos_list[i] != -1:
        #find barcode/start combo
        bc_start = find_barcodes(reads[i])
        #add to list 
        barcode_list.append(bc_start) #list will essentially be sorted by read start because reads are sorted?
        barcode_list.sort(key=itemgetter(0))  #maybe this will speed up search by barcode?
        # if hasn't been added before, add read to good_reads
        if barcode_list.count(bc_start) < 2:
          good_reads.append(reads[i])
        else:
          bad_reads.append(reads[i])
      else:
        notags_reads.append(reads[i])
    print("number of good reads (with tag): ", len(good_reads))
    print("number of bad reads (with no tag): ", len(notags_reads))
    print("Number of reads with duplicate barcode/starts: ", len(bad_reads))
    bn = os.path.basename(sam_file)
    outfile = "tag_filtered_" + bn
    with open(outfile, 'w') as f:
      for line in header:
        f.write(f"{line}\n")
      for line in good_reads:
        f.write(f"{line}\n")
  else:
    sys.exit("reads file length doesn't match position file")
  
  return barcode_list

In [70]:
filter_mapped_reads("sorted_reads/test_5000_R1.sam")

Number of reads with no transposon tag:  611
Number of reads with tag within first 32 bases:  635
number of good reads (with tag):  603
number of bad reads (with no tag):  611
Number of reads with duplicate barcode/starts:  32


[('AAAAACAG', '274566'),
 ('AAAAACGC', '3897925'),
 ('AAAACTAG', '224546'),
 ('AAAAGTCT', '3648873'),
 ('AAAATGTA', '1384138'),
 ('AAAATTAA', '3373285'),
 ('AAACAAAG', '1165185'),
 ('AAACAAGT', '475611'),
 ('AAACATTG', '3973761'),
 ('AAACCTTT', '2558063'),
 ('AAACTTCC', '985131'),
 ('AAAGCAAA', '2948421'),
 ('AAATAAAC', '0'),
 ('AAATAGTT', '1616431'),
 ('AAATAGTT', '1616431'),
 ('AAATCGCA', '2075169'),
 ('AAATGGAA', '3140968'),
 ('AACAACCA', '2773084'),
 ('AACAACCT', '3499990'),
 ('AACAATGC', '3143862'),
 ('AACACACA', '0'),
 ('AACACCTC', '1696057'),
 ('AACATACT', '1349442'),
 ('AACATTAC', '165903'),
 ('AACCAATC', '925154'),
 ('AACCAGAA', '87640'),
 ('AACTAACC', '0'),
 ('AACTATTC', '3859816'),
 ('AACTCCAA', '1775261'),
 ('AACTGTCA', '3905561'),
 ('AAGACATG', '2183427'),
 ('AAGACCAA', '2883723'),
 ('AAGACGCT', '837708'),
 ('AAGACGTC', '812555'),
 ('AAGATCAC', '307998'),
 ('AAGATGTA', '3847260'),
 ('AAGCAATA', '1480841'),
 ('AAGCCAGA', '1973979'),
 ('AAGCGCTT', '99975'),
 ('AAGGAGCA', '31

5. Filter for PCR amplification reads--reduce to single template counts (do at same time as finding transposon tag)

Will I be able to use the start position to do this?

Need to use new program since barcode now in different place.

1) function to identify barcode and start of each read
2) when found good_read --> apply function and compare barcode:start to list
3) if not in list --> append to good_reads, else to bad_reads

Need to parse sam flag--position is strand specific (sam flag is 2nd field (index=1) and 0 is forward, 16 is reverse). Start position is left-most position of alignment, so for forward it is the start, but for reverse it is the end. To get reverse start, have to add to position the length of the alignment. This is given in the CIGAR string (6th field, so index=5). 

reverse strand (samflag=16): 118M33S  so first number up to 'M' is length of match (33S is softclip of transposon seq)
forward strand (samflag=0): 33S118M so number AFTER S. But this wont need to be parsed, as left-most position is correct for forward.


In [9]:
#parse samflag 

def parse_samflag(read):
    """
    Function to determine strand orientation of alignment from read header
    """
    samflag = read.split()[1]
    if samflag=="16":
        strand = "R"
    elif samflag=="0":
        strand = "F"
    else:
        strand = "*"
    return strand

def align_len(read):

    """
    Function to determine read alignment length from start position of reads 
    filtered for transposon tag (should have soft-clipped ~30-33 before gDNA)

    Some cases may exist like 29S89M33S--transposon at both ends? or adapter not trimmed? 
    if forward, can ignore last part of alignment, if reverse, can ignore first part
    
    """
    import re
    cig = read.split()[5]
    cig_list = parse_cigar(cig)
    cig_last = len(cig_list) - 1
    strand = parse_samflag(read)
    if strand == "R":
        # assume alignment without extra bases at 3' end 
        #make sure no soft-clipping at start
        #if 'M' in cig_list[0]:  #this could be 3' end alignment
        if "S" in cig_list[cig_last]:  #check for soft-clipping (tag) at 5' end of read
            match_len = cig_list[cig_last - 1] #aligned part is next part
            #soft_clip = cig_list[cig_last]
            match_len = int(match_len.split("M")[0])
        else:
            match_len = 0
        
    elif strand == "F":
        #for F strand, softclipped bases at 3' end are cigar_list[2] and can be ignored?, if no soft-clipping at 5' end, will be no tag
        match_len = cig_list[1]
        match_len = int(match_len.split("M")[0])
        #soft_clip = cig_list[0]
    else:
        match_len = 0
    return match_len

def rev_start(pos, alignlen):
    """
    Function to find start position for reverse aligned reads
    """
    # need to subtract one from start to match actual start position of read on + strand
    #  and -1 for zero-index adjustment when adding
    rev_start = int(pos) + alignlen - 2
    return rev_start

def parse_cigar(cigar):
    """
    Parse cigar string 
    regex: \*|([0-9]+[MIDNSHPX=])+
    """
    import re
    #keep delimiter
    #cig_list = re.split('([0-9]+M|[0-9]+S)', cigar)
    #remove empty strings
    #cig_list = [i for i in cig_list if i]

    #find all number/letter combos in cigar string
    cig_list = re.findall(r'[0-9]+[MIDNSHPX=]+', cigar)
    return cig_list

In [106]:
#look at align lengths that are less than 50, all aligned lengths > 31

from Menadione_tnseq.ipynb:pylance-notebook-cell:Y144sZmlsZQ== import read_samfile


sf = read_samfile("sorted_reads/mapped_test_5000_R1.sam")[1]
for read in sf:
    x = align_len(read)
    print(x)
  
    # x_al = x[0]
    # x_sc = x[1]
    # sc_num = int(x_sc.split("S")[0])
    # if sc_num < 30:
    # #if "S" not in x_sc:
    #     print(x_al, x_sc)
    #     print(read)
    

118
119
118
118
118
109
118
95
118
69
118
79
118
118
118
118
118
121
121
116
119
118
118
118
118
118
118
118
118
118
119
119
72
118
119
118
113
118
114
118
88
118
118
118
118
118
118
118
89
89
118
118
119
118
118
118
119
118
118
118
118
118
118
119
119
85
118
118
87
118
118
118
118
118
118
118
118
118
111
85
118
118
86
94
118
110
118
107
118
119
118
48
93
101
118
87
93
118
119
118
93
118
118
118
118
119
118
118
118
118
118
47
116
116
92
59
34
118
31
118
118
118
118
109
118
118
119
118
118
75
118
101
122
118
118
118
118
91
118
118
118
118
118
118
118
120
118
107
118
118
119
35
118
118
118
118
118
119
119
118
118
118
118
118
118
67
118
118
113
118
118
118
118
118
109
118
118
53
97
118
85
119
105
118
85
118
118
71
77
118
119
118
118
59
118
118
118
118
115
118
118
118
118
77
62
118
37
118
77
118
119
118
118
70
118
118
73
118
118
42
45
118
118
121
118
118
118
54
118
118
80
119
59
112
118
118
113
102
75
118
118
118
118
118
118
118
118
47
118
118
118
118
118
119
114
117
118
118
104
102
52
102

All aligned lengths are greater than 31. All the ones checked appear to have transposon tag in the soft-clipped region. Could it be more efficient to just check this region for transposon tag in order to filter? This is less reliable because instead of sc, sometimes get D or other tags. Need to check later on that these aren't causing problems with align-length

A01968:63:H77VYDSX5:4:1101:3314:1626_1:N:0:AACGTGAT_BC:ATCGTGAT	16	NC_002945.3	3002949	60	78M1D40M33S	*	0	0	GCGCAGGTGTTGAACACCACGACGTCGGCCTCGGAACCGTCGGTCGCCCTCCGGTAGCCGGCCGCTTCCAGCAGACCCCCAGCCGCTCGGAGTCGTGGACGTTCATCTGACAGCCG*TAACAGGTTGGCTGATAAGTCCCCGGTCTCTAGAC	

In this case, on reverse strand, we want the most 5' aligned region to get start, so this will be closest to the soft clip. So we need to indicate the len(cigar_list)-1 to be the align len, NOT the 1st field. Have tested, and it works for this case, but this is only case in test file that has the D

Shorter sc seem to indicated truncated transposon without the -GTTA, but is this reliable for narrowing search region for tags?

In [164]:
def read_samfile(samfile):
    """
    read sam_file and sort lines between header and reads
    """

    header = []
    reads  = []

    with open(samfile, 'r') as f:
        for line in f:
            line = line.strip()
            if line[0] == "@":
                header.append(line)
            else:
                reads.append(line)
    return header, reads

In [107]:
heads = read_samfile("sorted_reads/mapped_test_5000_R1.sam")[1]
for read in heads:
    sf = parse_samflag(read)
    if sf not in ["R", "F"]:
        print(sf)
        print(read)
        al = align_len(read)
        print(al)
# position = heads.split()[3]
#bc_start = find_barcodes(heads)
#print(bc_start)
# rs = rev_start(position, al)
# print(rs)




re-do find_barcodes, find_tags and filter_mapped_reads to incorporate these functions and find correct start positions. Alignment length can be used to show if transposon tag in first part of read--aligned length needs to be >110 for it to be in first/last 40 bases regardless of strand (unlikely to be at end of aligned read)

In [172]:
def mmfind1(G,n,H,m,max): # lengths; assume n>m
  """
  A function to find matches of transposon sequence in read
    Input               G                           sequence string
                        n                           length of read sequence
                        H                           pattern string
                        m                           length of pattern
                        max                         maximum mismatches
    Output              i, -1                       start position of match in sequence, or -1 for no match
  """

  a = G[:n].find(H[:m])
  if a!=-1: return a # shortcut for perfect matches
  for i in range(0,n-m):  # range of 0 to difference between seq length and len pattern
    cnt = 0
    for k in range(m):
      if G[i+k]!=H[k]: cnt += 1
      if cnt>max: break
    if cnt<=max: return i
  return -1

def find_barcodes_starts(read):
    """
    Find barcode and read alignment start position
    Returns tuple of barcode and read start pos
    """
    read_name  = read.split()[0]
    barcode    = read_name.split("BC:",1)[1]
    rd_start = find_start(read)
    bar_start = (barcode, rd_start) 
    return bar_start


def find_start(read):
    """
    Parse read for samflag (strand), start of read position and cigar string for alignment length
    Returns aligned start position. Use this on filtered 'good reads' to identify start for quantification
    """
    strand   = parse_samflag(read)
    al_len   = align_len(read)
    left_pos = read.split()[3]
    if al_len > 0 and strand != "*":
      if strand == "F":
        read_start = int(left_pos)
      else:
        read_start = rev_start(left_pos, al_len)
    else:
        read_start = "-"
    return read_start

def check_seq(str):
    """
    Is the target tag made of acceptable nucleotide characters
    """
    import re
    chars = set('ACTG')
    seq = set(str.upper())
    if seq.issubset(chars):
        return True
    else:
        return False

def find_tags2(read, target_tag, max):
    """
    Find transposon tag in sequence of each read.

      Input           read              mapped read
                      target_tag        string that matches transposon sequence
                      max               maximum number of mismatches allowed (default=2)
      Output          match             match of start of tag from left-most start position of read or -1 if no match
    """
    from Bio.Seq import Seq
    
    # find strand of read
    strand = parse_samflag(read)
    seq = read.split()[9]
    if strand == "F":
        #search string for transposon seq with max num mismatches
        match = mmfind1(seq, len(seq), target_tag, len(target_tag), max)
    elif strand == "R":
        # make rc of seq and search for transposonl tag with max num mismatches
        dna     = Seq(seq)
        rc_seq  = str(dna.reverse_complement())
        match = mmfind1(rc_seq, len(rc_seq), target_tag, len(target_tag), max)
    else: 
        match = -1
    
    return match

def filter_mapped_reads2(sam_file, tag="ACTTATCAGCCAACCTGTTA", mismatch_max=2):
  """ 
  Revised filtering function
  """
  import sys
  import os
  import re
  from operator import itemgetter

  #check tag has valid nucleotides
  if check_seq(tag):
     pass
  else:
     sys.exit("Invalid sequence tag")

  #read sam_file and sort lines between header and reads
  data    = read_samfile(sam_file)
  header  = data[0]
  reads   = data[1]

  barcode_list  = []
  good_reads    = []
  notags_reads  = []
  dup_reads     = []
  
  for read in reads:
    ftag = find_tags2(read, tag, mismatch_max)
    #align length (will parse read align len from header)
    al  = align_len(read)

    if ftag != -1 and al > 0:   #(transposon tag in first 40, but this will discard shorter reads that have tag and have mapped, could make 0)
      #find barcode/start combo
      bc_start = find_barcodes_starts(read)
      barcode_list.append(bc_start) 
      barcode_list.sort(key=itemgetter(0))  #maybe this will speed up search by barcode?
     
      # if hasn't been added before, add read to good_reads
      if barcode_list.count(bc_start) < 2:
        good_reads.append(read)
      else:
        dup_reads.append(read)

    else:
      notags_reads.append(read)
  #save barcode_list as pickle object to use in quantifying reads at ta sites? temporary until read processed?  
  print("Total number of mapped reads: ", len(reads))
  print("number of good reads (with tag): ", len(good_reads))
  print("number of bad reads (with no tag): ", len(notags_reads))
  print("Number of reads with duplicate barcode/starts: ", len(dup_reads))
  bn = os.path.basename(sam_file)
  outfile = "tag_filtered_" + bn
  with open(outfile, 'w') as f:
    for line in header:
      f.write(f"{line}\n")
    for line in good_reads:
      f.write(f"{line}\n")
  return notags_reads

Have filtered out unmapped reads from sorted.bam into mapped.sam. All reads should be assigned a strand in this case

In [110]:
def check_seq(str):
    import re
    chars = set('ACTG')
    seq = set(str.upper())
    if seq.issubset(chars):
        return True
    else:
        return False

print(check_seq("ACTGGggG"))


True


In [159]:
my_reads = read_samfile("sorted_reads/mapped_test_5000_R1.sam")[1][0:10]
#for read in my_reads:   
#    print(read)
 #   fb = find_barcodes2(read)
 #   print(fb)
 #   tg = find_tags2(read, "ACTTATCAGCCAACCTGTTA", 2)
 #   print(tg)
    
filter_mapped_reads2("sorted_reads/mapped_test_5000_R1.sam")


Total number of mapped reads:  1180
number of good reads (with tag):  1105
number of bad reads (with no tag):  7
Number of reads with duplicate barcode/starts:  68


['A01968:63:H77VYDSX5:4:1101:5267:1752_1:N:0:AACGTGAT_BC:GGGGGGGG\t16\tNC_002945.3\t186581\t60\t119M32S\t*\t0\t0\tCGCTGTTCTACGCCGACGGCACCACTATGTTGTTCGGTGATGCGAAGAAATCGGTTACCGAAGTCTCCGAGGAACTCAAGGCGTTGTAGCGCGCGAGCGCTGGCTCAGACGGGCGGATAACAGGTTGGCTGATAGTCCTCGGTCTCTAGAC\tF:FFF,FFFFFFFFFFFFFFF:FFF::FFFFFFFF:FFFFFF,FFFFFFFFF:FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF:FFFFFFFFFFFFFFF:FFFFF:FFFFF:FFFFFFFFFFF:FFFFFF:\tNM:i:1\tMD:Z:55G63\tAS:i:114\tXS:i:0',
 'A01968:63:H77VYDSX5:4:1101:21766:2300_1:N:0:AACGTGAT_BC:TATTTCAT\t16\tNC_002945.3\t215396\t60\t119M32S\t*\t0\t0\tTGCAGCTCTTCGTGCTCATCGCCGCGGCCCACGACGTGCGCTGCGACGTGGCATCGAATTCGCCGTTCGTGTACGCCTACGGGTTCGCCGAGGACATCGACACCAGCCACGCCCTATACCCCGTGGGCTGATAAGTCCCCGGTCTCTAGAC\tF::FF:F,F,:FFFF:,F:F,F,FFF,F,:F:FFF:,FFFF:,,FFFF,FFFFFFFFFFFF::FF,:F:FFF,FF:F::FFF::FFFFFF,FFF,FFFF,FFFF:,FF:FFFF:F:,F,F,,FF,,FFFF:FFFFFFFF:FFFFFFFFFFF\tNM:i:3\tMD:Z:22G6A32A56\tAS:i:104\tXS:i:0',
 'A01968:63:H77VYDSX5:4:1101:11559:2002_1:N:0:AACGTGAT_BC:ACACCACT\t16\tNC_0

6 of these 7 no tag reads have different part of the transposon tag in them, but not the part that ends in GTTA

+  ACTTATCAGCCAACCTGTTA-gDNA
-  gDNA-TAACAGGTTGGCTGATAAGT
            gDNA?-GCTGATAAGTCCCCGGTCTCTAGAC
 

These have parts of transposon tag, but either >2 matches, or missing the terminal end of Himar1 transposon which includes the TA site.

View these on artemis. Artemis won't view .sam, so have to change to .bam

In [ ]:
conda activate dna
samtools view -b -o tag_filtered_mapped_test_5000_R1.bam tag_filtered_mapped_test_5000_R1.sam

This samtools view won't work with the filtered because the @SQ lines are removed when filtered for mapped reads.

Can use -h to reset output with header--edited snakemake file for filtering by mapped reads.



6. Quantify reads at each TA site (need list of coordinates of TA sites for bovis to make wig files). Need to make sure start position parsed from the cigar string is accurate

How do left-most positions from reads align with TA sites?



## View on artemis

Reverse strand mapped read with start position calculated as 39,859 (left-most position + aligned len from cigar string). This is actually one off from real start position which is one too far right (5' for reverse). REad actually begins at 39858. TA site on - strand starts at 39858. Positive read starts at 39857 (correct left-most position from cigar) which coincides with TA starting at 39857. These are same TA site and should be counted as 2 insertions for one TA site. The site is referenced by + strand T of the TA in the file of ta sites (bovis_TA_sites.txt) found with insertion_site_finder function. Therefore, reverse strand reads need to subtract one from + strand coordinate of start to line up with the correct TA site (which is really the 'A' of the TA site on the reverse compl strand)

![Forward and reverse mapped to same TA site](images/F_R_sameTAsite.png)


For forward strand:  start == TA site position (+ strand coordinate)
for reverse strand:  start - 1 == TA site position (based on + strand coordinate) (this is in addition to -1 to calculate the start position from align len, subtract 2 from start in find_start/rev_start functions)

Instead of changing start position for reverse reads, will quantify strands separately, with TA sites adjusted for reverse orientation? This causes problems when comparing lists--if starts are integers and ta_sites are strings. do all adjustments at start assignment BEFORE quantification

Also, when TA site inserted before T in genomic, alignment len can be off by one (should have soft-clipped one more base as genomic DNA seq has TTA which matches transposon tag end -GTTA), should there be some leeway if start is within 1 base of TA site position?

![soft-clip one early](images/softclip_error.png)

Maybe can fix this when finding transposon tag--if there is a match, change start to end position of match? (returned value + transposon tag length). may be more accurate than start left most start of alignment? can still add alignment length to this for reverse reads. Essentially, "virtually" clip transposon when determining start site.

In [220]:


def motif_finder(seq, motif):
    """Finds location of motif (substring) in sequence.
    Input       seq         sequence
                motif       desired subsequence
    Output      locations   list of sequence locations of substrings (one-indexed)
    """
    import re
    locations = []
    p = re.compile('(?='+motif+')')       # use lookahead to find all overlapping matches
    matches = p.finditer(seq)
    for match in matches:
        start = match.span()
        locations.append(start[0] + 1)   # add one to change from zero-indexing to sequence position   


    return locations

#**********************************************************************************

def find_insertion_sites(seq):
    """Finds all the locations of TA insertion sites in sequence

    Seems you only want TA positions from one strand for counting number of sites,
    as they are same site on either strand. When quantifying, need to get reads that start 
    """
    from Bio.Seq import Seq
    
    seq_obj         = Seq(seq)
    rec_seq         = str(Seq.reverse_complement(seq_obj))
    fwd_positions = motif_finder(seq, "TA")
    rev_positions = motif_finder(rec_seq, "TA")
    
    return fwd_positions

#**********************************************************************************


def open_fasta(refseq):
    with open(refseq, 'r') as file:
        seq = ''
        for line in file:
            if line[0] != '>':
                seq += line.strip()
    return seq
    


Make list of TA sites in reference genome

In [222]:
bovis_fasta = "ref_seqs/Mbovis_AF2122-97.fasta"
bovis_seq   = open_fasta(bovis_fasta)
# to look at only first 30,000 nts of genome
bovis_30 = bovis_seq[0:30001]

res = motif_finder(bovis_seq, "TA")
print(len(res))
print(res[0:10])

with open('bovis_TA_sites.txt', 'w') as outfile:
    results = find_insertion_sites(bovis_seq)
    #print(len(results))
    #73465
    #print(results[0:10])
    #print(len(results[1]))
    #73465
    #these will be from 5' end of rc strand (-), wont use these
    #print(results[1][0:10])
    #these should coincide with 5' end of + strand
    #print(results[1][-10:])
    #for site in results:
    print(results, file=outfile)


    


73465
[60, 72, 102, 188, 246, 333, 360, 426, 448, 471]


In [163]:
whole_set = [1,2,3,4,5,5,5,5,6,7,8,9,10]
print(len(whole_set))
unique_set = set(whole_set)
print(unique_set)
len(unique_set)

13
{1, 2, 3, 4, 5, 6, 7, 8, 9, 10}


10

Use similar strategy to tpp_tools.py: read_counts(ref,sam,vars) template_counts(ref,sam,bcfile,vars) and increase_counts(pos,sites, strand)

1. use find_barcode_starts to get list of barcode/starts for each read. should be same length as number of reads
    - change filter function to only look for tranposon tags
    - new function to identify starts/strand/barcode for each read and make into list(which will be iterated through)
2. make dicts of ta-sites and read counts: ordered dictionary (default is ordered), new values will overwrite old values
    - analyse each strand separately
    - make TA:read dict: 
        - for each site key, append list barcodes that have starts within 1 nt of TA site
        - add up total reads for each site (F and R)
    - make TA:template dict:
        - for each site key, reduce barcode list to unique sites
        - add up total reads for each site (F and R)
3. Write wig files for TA sites using TA:template dict




In [173]:
#make new dictionary with start positions and strand? or re-use barcode/start--add strand info?



def find_barcodes_starts_strand(reads):
    """
    Find barcode and read alignment start position
    Returns list of starts, strand, barcode
    Input           read
    Output          starts_list
    """

    starts_list = []
    for read in reads:
        read_name  = read.split()[0]
        barcode    = read_name.split("BC:",1)[1]
        strand     = parse_samflag(read) 
        rd_start   = find_start(read)
        line = [rd_start, strand, barcode]
        starts_list.append(line)
    return starts_list


In [178]:
my_reads = read_samfile("sorted_reads/mapped_test_5000_R1.sam")[1] #reads are second in return

mlist = find_barcodes_starts_strand(my_reads)
print(len(mlist))
for x in mlist[0:20]:
    print(x)

print(mlist[0][1])



1180
[20350, 'R', 'GAACAACT']
[26121, 'F', 'CGCCACCT']
[31581, 'F', 'CCGACTGA']
[35236, 'R', 'TATTCTAA']
[37337, 'F', 'AATCACTC']
[39859, 'R', 'TGAGGCGA']
[39857, 'F', 'CGAGACAG']
[47375, 'F', 'ACAGGATC']
[48094, 'R', 'CTGCTCCA']
[51951, 'F', 'GACCAAGC']
[52823, 'R', 'CATAGAAT']
[57868, 'F', 'CAGTGTAC']
[63369, 'R', 'TTACTATT']
[69581, 'R', 'TTTAGTCA']
[72388, 'R', 'TTACCATA']
[72388, 'R', 'TTACCATA']
[81203, 'R', 'ATATTCCT']
[81968, 'R', 'AAGGTTAA']
[81968, 'R', 'AAGGTTAA']
[87311, 'F', 'TGTTCCAG']
R


Iterate through list and add to both dictionaries



In [258]:
def quantify_sites(ta_sites, start_sites_list):
    """
    Function to quantify reads at each ta site
    """

    template_dict   = {}
    #rev_template_dict   = {}
    no_match        = []
    read_count_dict     = {}

    for site in start_sites_list:
        #print(site)
        start_site  = site[0]
        #print(start_site)
        strand      = site[1]
        #print(strand)
        barcode     = site[2]
        # add to read dict
        
        # do separately for strands because easier to demultiplex (different reads if on different strands by def)
        #if strand == 'F':
        if str(start_site) in ta_sites:
            site_list = template_dict.get(start_site, None)
            print(site_list)
            if site_list is not None:  #there are already entries at this site
                read_count_dict[start_site] = read_count_dict[start_site] + 1
                    #if barcode != site_list: #ie barcode is different than those in list 
                site_list.append(site)
                template_dict[start_site] = site_list
                    #else: 
                    #    pass
            else: #no entries with this site key, create new one
                read_count_dict[start_site] = 1
                site_list = site
                template_dict[start_site] = site_list
        else:
            no_match.append(site)

        # if strand == 'R':
        #     if str(start_site) in ta_sites:
        #         site_list = rev_template_dict.get(start_site, None)
        #         if site_list is not None: 
        #             read_count_dict[start_site] = read_count_dict[start_site] + 1
        #             #if site not in site_list:
        #             site_list.append(site)
        #             rev_template_dict[start_site] = site_list
        #             #else:
        #             #    pass
        #         else:
        #                 read_count_dict[start_site] = 1
        #                 site_list = site
        #                 rev_template_dict[start_site] = site_list
        #     else:
        #         no_match.append(site)
    
    

    return template_dict, read_count_dict, no_match


In [260]:
my_reads = read_samfile("sorted_reads/mapped_test_5000_R1.sam")[1][0:20] #reads are second in return
print(len(my_reads))
mlist = find_barcodes_starts_strand(my_reads)
with open('bovis_TA_sites.txt', 'r') as f:
    ta_list = f.read()
    
test_dict = quantify_sites(ta_list, mlist)

print("templates", test_dict[0])
#print("reverse_templates", test_dict[1])
print("read counts for all sites:" , test_dict[1])
print("no_match", test_dict[2])



20
None
None
None
None
None
None
[39857, 'R', 'TGAGGCGA']
None
None
None
None
None
None
None
None
[72386, 'R', 'TTACCATA']
None
None
[81966, 'R', 'AAGGTTAA']
None
templates {20348: [20348, 'R', 'GAACAACT'], 26121: [26121, 'F', 'CGCCACCT'], 31581: [31581, 'F', 'CCGACTGA'], 35234: [35234, 'R', 'TATTCTAA'], 37337: [37337, 'F', 'AATCACTC'], 39857: [39857, 'R', 'TGAGGCGA', [39857, 'F', 'CGAGACAG']], 47375: [47375, 'F', 'ACAGGATC'], 48092: [48092, 'R', 'CTGCTCCA'], 51951: [51951, 'F', 'GACCAAGC'], 52821: [52821, 'R', 'CATAGAAT'], 57868: [57868, 'F', 'CAGTGTAC'], 63367: [63367, 'R', 'TTACTATT'], 69579: [69579, 'R', 'TTTAGTCA'], 72386: [72386, 'R', 'TTACCATA', [72386, 'R', 'TTACCATA']], 81201: [81201, 'R', 'ATATTCCT'], 81966: [81966, 'R', 'AAGGTTAA', [81966, 'R', 'AAGGTTAA']], 87311: [87311, 'F', 'TGTTCCAG']}
read counts for all sites: {20348: 1, 26121: 1, 31581: 1, 35234: 1, 37337: 1, 39857: 2, 47375: 1, 48092: 1, 51951: 1, 52821: 1, 57868: 1, 63367: 1, 69579: 1, 72386: 2, 81201: 1, 81966: 2,

sites close to these:  134438 (F -1), 154950 (F -1), 163066 (R +4), 215512 (510) (R -1), 231802 (806,808) (F +1)
248316 (R -1), 253846 (F +1)

Barcode/template dedup not working here--too complicated. 
1. find more accurate start position using position of transposon tag (will eliminate some of the nomatches to ta sites that are off by a base or two)
2. deduplicate barcode/start as before so quantification step using already deduped reads. won't have total read count, but will have count of deduped reads for each barcode/start?
    - use find_barcodes_starts

Alternatively, use genomic ranges in R to detect overlap between start of read and TA site with some flexibility in the read start.

New filtering by transposon tag that uses location of tag to define start


In [376]:
def find_ta_position(read, tag_pos):
    """
    Find the TA position relative to + strand
    """
    strand = parse_samflag(read)
    left_pos = read.split()[3]
    if tag_pos != -1:
        if strand == 'F':
            #TA should be the left most position == soft-clipped bases + tag_pos
            ta_position = int(left_pos)
        else:
            ta_position = int(left_pos) + tag_pos
    else:
        ta_position = "-"
    return ta_position


In [377]:
def find_tags2(read, target_tag, max):
    """
    Find transposon tag in sequence of each read.

      Input           read              mapped read
                      target_tag        string that matches transposon sequence
                      max               maximum number of mismatches allowed (default=2)
      Output          start             calculation of start of read/insertion point 
                                        from left-most start position of tag or -1 if no match
    """
    from Bio.Seq import Seq
    #find rec of tag
    tag_seq = Seq(target_tag)
    rc_tag  = str(tag_seq.reverse_complement())
    # find strand of read
    strand = parse_samflag(read)
    seq = read.split()[9]
    if strand == "F":
        #search string for transposon seq with max num mismatches
        match = mmfind1(seq, len(seq), target_tag, len(target_tag), max)
        if match != -1:
            start = match + len(target_tag) #this gives start position of read (start of ta site)
        else:
            start = match
    else: #strand is reverse
        #this is too time consuming to find rc for each read
        # make rc of seq and search for transposon tag with max num mismatches ()
        #dna     = Seq(seq)
        ##rc_seq  = str(dna.reverse_complement())
        #rev_match   = mmfind1(rc_seq, len(rc_seq), target_tag, len(target_tag), max)
        #match   = len(seq) - rev_match #gives match coordinate (start of tag)
        match = mmfind1(seq, len(seq), rc_tag, len(rc_tag), max)
        if match != -1:
            start = match  #to get to start of 'TA' in reverse read
        else: 
            start = match
    
    return start

In [378]:
my_reads = read_samfile("sorted_reads/mapped_test_5000_R1.sam")[1][0:20] #reads are second in return
for read in my_reads:
    x = find_tags2(read, "ACTTATCAGCCAACCTGTTA", 2)
    print(x)
    print(find_ta_position(read, x))
    print(read)


116
20348
A01968:63:H77VYDSX5:4:1101:8757:2503_1:N:0:AACGTGAT_BC:GAACAACT	16	NC_002945.3	20232	60	118M33S	*	0	0	ATACGCGTTCGATGACCTCGGTGCCGGCCGCCGTAATCGGCGACTTATTTCGTGGGCGGGTGCGCAGTGGGCGGCGGGCTCCGTGCGAGATGCGTGCCAGGATGGCCAGCAATATGTAACAGGTTGGCTGATAAGTCCCCGGTCTCTAGAC	FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF,FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF	NM:i:0	MD:Z:118	AS:i:118	XS:i:0
35
26121
A01968:63:H77VYDSX5:4:1101:15022:1611_1:N:0:AACGTGAT_BC:CGCCACCT	0	NC_002945.3	26121	60	32S119M	*	0	0	GTCTAGAGACCGGGGACTTATCAGCCAACCTGTTAGCACGCTCATCGTGTGTCCTTGCGGCCAGGGATAGCGCCGTAGCTGATCGTAGATAGTGGTGCGGCACATGTCGTCGCCAGTGGCCGCCAGCAGCGGGTCCCGCGCCTGCGGTGTG	FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF	NM:i:0	MD:Z:119	AS:i:119	XS:i:0
35
31581
A01968:63:H77VYDSX5:4:1101:20184:2942_1:N:0:AACGTGAT_BC:CCGACTGA	0	NC_002945.3	31581	60	33S118M	

In [421]:
def filter_mapped_reads3(sam_file, tag="ACTTATCAGCCAACCTGTTA", mismatch_max=2):
  """ 
  Revised filtering function
  """
  import sys
  import os
  from operator import itemgetter

  #check tag has valid nucleotides
  if check_seq(tag):
     pass
  else:
     sys.exit("Invalid sequence tag")

  #read sam_file and sort lines between header and reads
  data    = read_samfile(sam_file)
  header  = data[0]
  reads   = data[1]


  barcode_list    = []
  template_reads  = []
  template_barcodes = []
  notags_reads    = []
  dup_reads       = []
  
  for read in reads:
    tag_pos   = find_tags2(read, tag, mismatch_max)
    strand    = parse_samflag(read)
    
    #align length (will parse read align len from header)
    #al      = align_len(read)
    #if strand == "F":

    if tag_pos != -1:  #there is match for tranposon tag
        insertion_coord = find_ta_position(read, tag_pos)
        #find barcode
        read_name  = read.split()[0]
        barcode    = read_name.split("BC:",1)[1]
        pos_barcode = [insertion_coord, strand, barcode]
        #add to list of all reads with insertions
        barcode_list.append(pos_barcode) 
        barcode_list.sort(key=itemgetter(0))  #maybe this will speed up search ordering by position--but sorting will add time
        # if hasn't been added before, add read to unique template reads
        if barcode_list.count(pos_barcode) < 2:
          template_reads.append(read)
          template_barcodes.append(pos_barcode)
        else:
          dup_reads.append(read)
    else:
      notags_reads.append(read)
  
  print("Total number of mapped reads: ", len(reads))
  print("number of unique templates (with tag): ", len(template_reads))
  print("number of bad reads (with no tag): ", len(notags_reads))
  print("Number of reads with duplicate barcode/starts: ", len(dup_reads))
  bn = os.path.basename(sam_file)
  #write template reads
  outfile = "tag_filtered_templates_" + bn
  with open(outfile, 'w') as f:
    for line in header:
      f.write(f"{line}\n")
    for line in template_reads:
      f.write(f"{line}\n")
  outfile_dups = "duplicate_reads_" + bn
  with open(outfile_dups, 'w') as f:
    for line in header:
      f.write(f"{line}\n")
    for line in dup_reads:
      f.write(f"{line}\n")
  outfile_notag = "notag_filtered_mapped_" + bn
  with open(outfile_notag, 'w') as f:
    for line in header:
      f.write(f"{line}\n")
    for line in notags_reads:
      f.write(f"{line}\n")

  return template_barcodes

In [422]:
samfile = "sorted_reads/mapped_test_5000_R1.sam"
template_list = filter_mapped_reads3(samfile)
#check that unique templates are actually unique
template_barcodes = []
for line in template_list:
    template_barcodes.append(line[0])
    
print(len(template_barcodes))
unique_barcodes = set(template_barcodes)
print(len(unique_barcodes))

# for x in set(template_barcodes):
#     count_x = template_barcodes.count(x)
#     if count_x >1:
#         print(x, count_x)

#print([[x,template_barcodes.count(x)] for x in set(template_barcodes)])

print(len(template_list))
template_set = set()
for item in template_list:
    template_set.add(str(item))
print(len(template_set))


Total number of mapped reads:  1180
number of unique templates (with tag):  1107
number of bad reads (with no tag):  7
Number of reads with duplicate barcode/starts:  66
1107
1043
1107
1107


There are duplicate barcodes in templates--do these have different starts/strands? All unique combinations.

HOw many of these starts are in TA-site list?

In [384]:


def take_closest(myList, myNumber):
    """
    Assumes myList is sorted. Returns closest value to myNumber.

    If two numbers are equally close, return the smallest number.

    from https://stackoverflow.com/questions/12141150/from-list-of-integers-get-number-closest-to-a-given-value/12141511#12141511
    """
    from bisect import bisect_left
    pos = bisect_left(myList, myNumber)
    if pos == 0:
        return myList[0]
    if pos == len(myList):
        return myList[-1]
    before = myList[pos - 1]
    after = myList[pos]
    if after - myNumber < myNumber - before:
        return after
    else:
        return before

In [27]:
import scripts.tnseq_pro as tn
samfile = "tests/mapped_test_5000_R1.sam"
bovis_fasta = "ref_seqs/Mbovis_AF2122-97.fasta"
bovis_seq   = tn.open_fasta(bovis_fasta)
ta_sites = tn.find_insertion_sites(bovis_seq)
results = tn.filter_mapped_reads3(samfile)
barcode_list = results

print("Number of reads with tags (total): ", len(barcode_list))
#print(barcode_list[0:5])

count = 0
for line in barcode_list:
    insert_start = line[0]
    #closest_ta = min(ta_sites, key=lambda x:abs(x-insert_start))  #too slow
    closest_ta = take_closest(ta_sites, insert_start)
    if abs(closest_ta - insert_start) < 2:
        count +=1
    else:
        print(line)
print("number of starts found in TA site list: ", count)



Total number of mapped reads:  1180
number of unique templates (with tag):  1107
number of bad reads (with no tag):  7
Number of reads with duplicate barcode/starts:  66
Number of reads with tags (total):  1107


NameError: name 'take_closest' is not defined

This gives ok results--1113 reads mapped to a TA site out of 1180 reads total and 1107 unique templates, so seems all templates are mapped to ta site.

Need to use the 'take_closest' function to quantify sites

In [435]:

def assign_counts_to_sites(ta_sites, barcode_position_list):
    """
    Function to tally ta sites with insertions
    """

    insertions_list = [line[0] for line in barcode_position_list]
    read_count_dict = {}
    no_match        = []

    for site in insertions_list:
        #check if ins_site is in list of ta sites (or within 1 nt of ta position)
        closest_ta = take_closest(ta_sites, site)
        if abs(closest_ta - site) < 2:
            if closest_ta not in read_count_dict:
                read_count_dict[closest_ta] = 1
            else:
                read_count_dict[closest_ta] += 1
        else:
            no_match.append(site)

    return read_count_dict, no_match

In [436]:
samfile = "sorted_reads/mapped_test_5000_R1.sam"
bovis_fasta = "ref_seqs/Mbovis_AF2122-97.fasta"
bovis_seq   = open_fasta(bovis_fasta)
ta_sites = find_insertion_sites(bovis_seq)
# make list of the positions of every insertion from unique templates
template_positions = filter_mapped_reads3(samfile)
print(len(template_positions))
#count number of reads per ta site
res=assign_counts_to_sites(ta_sites, template_positions)
print(len(res[0]))
print(res[0])



Total number of mapped reads:  1180
number of unique templates (with tag):  1107
number of bad reads (with no tag):  7
Number of reads with duplicate barcode/starts:  66
1107
988
{20348: 1, 26122: 1, 31581: 1, 35234: 1, 37337: 1, 39857: 2, 47375: 1, 48092: 1, 51951: 1, 52821: 1, 57868: 1, 63367: 1, 69579: 1, 72386: 1, 81201: 1, 81963: 1, 87311: 1, 87650: 1, 87640: 1, 99975: 1, 105899: 1, 108988: 1, 121806: 1, 122140: 1, 122870: 1, 126866: 1, 129020: 1, 134438: 1, 138138: 1, 154950: 1, 159782: 1, 160624: 1, 162598: 1, 163066: 1, 165864: 1, 165903: 1, 166239: 1, 169495: 1, 176268: 1, 176424: 1, 177547: 1, 178448: 1, 181212: 1, 186606: 1, 188101: 1, 190977: 1, 210469: 1, 222102: 1, 223131: 1, 223235: 1, 224546: 1, 225688: 1, 229915: 1, 231802: 1, 248316: 1, 253846: 1, 263856: 1, 268159: 1, 274566: 1, 274871: 1, 275021: 1, 280048: 1, 295715: 1, 300409: 1, 301037: 1, 307998: 1, 318230: 1, 321350: 1, 325804: 1, 329185: 1, 331336: 1, 337402: 1, 341987: 1, 361190: 1, 361207: 1, 363748: 1, 3642

In [450]:
#function to go through all possible ta sites and create .wig file of insertions

def ta_sites_to_dict(ta_sites, read_count_dict):
    """ 
    Function to create wig file of insertions
    Input               ta_sites                ordered list of all possible ta sites in Mbovis genome
                        read_count_dict         dictionary of ta sites with insertions and read counts
    Output              wig_dict                dict of insertions

    """
    wig_dict = {}
    for site in ta_sites:
        if site in read_count_dict:
            count = read_count_dict[site]
        else:
            count = 0
        wig_dict[site] = count
    return wig_dict

def write_wig_from_dict(wig_dict, sample_name, genome):
    """
    Function to write wig file from dict of insertions
    Input               wig_dict            dict of insertions
                        outfile_name        str of outfile name
    Output              outfile             wig file of insertions
    """
   
    #write template reads
    outfile = "output/" + sample_name + "_insertions.wig"
    with open(outfile, 'w') as f:
        f.write("#generated by tnseq_pro from " + sample_name + "\n")
        f.write("variableStep chrom=" + genome + "\n")
        for key, value in wig_dict.items():
            f.write(f"{key} {value}\n")
    return outfile


In [ ]:
samfile = "sorted_reads/mapped_test_5000_R1.sam"
bovis_fasta = "ref_seqs/Mbovis_AF2122-97.fasta"
bovis_seq   = open_fasta(bovis_fasta)
ta_sites = find_insertion_sites(bovis_seq)
# make list of the positions of every insertion from unique templates
template_positions = filter_mapped_reads3(samfile)
print(len(template_positions))
#count number of reads per ta site
res=assign_counts_to_sites(ta_sites, template_positions)
ta_dict = ta_sites_to_dict(ta_sites, res[0])
for key, value in ta_dict.items():
    if value > 0:
        print(key, value)

print(len(ta_dict))
write_wig_from_dict(ta_dict, "test", "Mbovis_AF2122-97")

In [445]:
#wrapper for creating wig file from sam file

def sam_to_wig(samfile, genome_fasta, sample_name):
    """
    Wrapper function to create wig file from sam file
    Input           samfile         sam file of mapped reads   
                    genome_fasta    path to fasta file of genome        str
                    sample_name     name of sample                      str
    Output          wig file        wig file of insertions
    """
    import os
    fasta       = genome_fasta
    #get genome name from fasta file
    genome      = os.path.basename(fasta).split(".")[0]
    fasta_seq   = open_fasta(genome_fasta)
    ta_sites    = find_insertion_sites(fasta_seq)
    # make list of the positions of every insertion from unique templates
    template_positions = filter_mapped_reads3(samfile)
    #count number of reads per ta site
    res=assign_counts_to_sites(ta_sites, template_positions)
    ta_dict = ta_sites_to_dict(ta_sites, res[0])
    wig_file = write_wig_from_dict(ta_dict, sample_name, genome)
    return wig_file

In [451]:
sam = "sorted_reads/mapped_test_5000_R1.sam"
bovis_fasta = "ref_seqs/Mbovis_AF2122-97.fasta"

sam_to_wig(sam, bovis_fasta, "test")

Total number of mapped reads:  1180
number of unique templates (with tag):  1107
number of bad reads (with no tag):  7
Number of reads with duplicate barcode/starts:  66


'output/test_insertions.wig'

## Map and test entire sample

moved scripts to thoth. 

In [ ]:
scp ~/tn_seq/menadione_tnseq/scripts/tnseq_pro.py sj003@ssh.cryst.bbk.ac.uk:/d/in16/u/sj003/men_tnseq/scripts/

scp ~/tn_seq/menadione_tnseq/Menadione_tnseq.ipynb sj003@ssh.cryst.bbk.ac.uk:/d/in16/u/sj003/men_tnseq/



Write script to apply add_barcode.py to each R1/R2 combination

Updating (tnseq_process_pipeline.ipynb) to summarise steps and code.

Uses module 'tnseq_pro.py'


In [ ]:
#iterate through samples and run sam_to_wig script

# iterate through files
import scripts.tnseq_pro as tn

# make list of .sam files in directory
import os
import glob
import re
sam_files = glob.glob("sorted_reads" + "/*.sam")
print(sam_files)
bovis_fasta = "ref_seqs/Mbovis_AF2122-97.fasta"
for file in sam_files:
    #find sample name from file
    sample_filename = os.path.basename(file).split(".")[0]
    sample_name = re.findall(r'mapped_(\w*)_R1_001', sample_filename)[0]
    print(sample_name)
    print(file)
    tn.sam_to_wig(file, bovis_fasta, sample_name)


On laptop, this takes a long time: stopped after 210 min for single file

this step seems to be the really long one: barcode_list.sort(key=itemgetter(0))  
Where it filters by barcodes and starts.
-find tag
-find start position
-eliminate duplicates for each start position

searching by TA site (only ~73K) and eliminate extra barcodes will be much faster

find_closest might also take a while...
Perhaps only run find closest on the ones that are don't match a ta site. Majority of those with tag matched a TA site.

### speed optimisation

filter_mapped_reads3(sam_file, tag, mismatch_max) is taking way too long, mainly at point where it has to sort all the reads by barcode.

break down into different parts with duplicates in final part--searching each ta site for duplicate barcodes

In [ ]:
import scripts.tnseq_pro as tn

def filter_mapped_reads_tag(sam_file, tag="ACTTATCAGCCAACCTGTTA", mismatch_max=2):
  """ 
  Revised filtering function: filters reads for presence of transposon tag and for duplicate barcode/ta-position

  Input             sam_file                            .sam file of mapped reads
                    tag                                 str of transposon tag used, default is Himar1
                    mismatch_max                        integer of maximum mismatches allowed in search for tag
  Output            read barcodes                       list of 3 values: insert_start, strand, barcode from reads with insertions
                                                        
  """
  import sys
  import os

  #check tag has valid nucleotides
  if tn.check_seq(tag):
     pass
  else:
     sys.exit("Invalid sequence tag")

  tagged_reads    = []
  barcode_list    = []
  notag_count = 0

  #read sam_file and sort lines between header and reads
  data    = tn.read_samfile(sam_file)
  header  = data[0]
  reads   = data[1]

  for read in reads:
    tag_pos   = tn.find_tags2(read, tag, mismatch_max)
    strand    = tn.parse_samflag(read)
    if tag_pos != -1:  #there is match for tranposon tag
        insertion_coord = tn.find_ta_position(read, tag_pos)
        #find barcode
        read_name  = read.split()[0]
        barcode    = read_name.split("BC:",1)[1]
        pos_barcode = [insertion_coord, strand, barcode]
        #add to list of all reads with insertions
        barcode_list.append(pos_barcode) 
        #append read to tagged reads
        tagged_reads.append(read)
    else:
      notag_count += 1
  
  print("Total number of mapped reads: ", len(reads))
  print("number of reads with tag: ", len(barcode_list))
  print("number of reads with no tag: ", notag_count)
  
  #write read files
  bn = os.path.basename(sam_file)
  outfile = "tag_filtered_templates_" + bn
  with open(outfile, 'w') as f:
    for line in header:
      f.write(f"{line}\n")
    for line in tagged_reads:
      f.write(f"{line}\n")
  
  

  return barcode_list

In [51]:
sam = "tests/mapped_test_5000_R1.sam"
bc_list = filter_mapped_reads_tag(sam)
print(bc_list[0:10])

Total number of mapped reads:  1180
number of reads with tag:  1173
number of reads with no tag:  7
[[20348, 'R', 'GAACAACT'], [26121, 'F', 'CGCCACCT'], [31581, 'F', 'CCGACTGA'], [35234, 'R', 'TATTCTAA'], [37337, 'F', 'AATCACTC'], [39857, 'R', 'TGAGGCGA'], [39857, 'F', 'CGAGACAG'], [47375, 'F', 'ACAGGATC'], [48092, 'R', 'CTGCTCCA'], [51951, 'F', 'GACCAAGC']]


In [53]:
def assign_sites(ta_sites, barcode_list):

    """
    Function to assign 
    Input               ta_sites                ordered list of all possible ta sites in Mbovis genome
                        site_barcode_dict       dictionary of ta sites:[list of all barcodes at site]
    Output              tasite_barcode_dict     dictionary of ta sites with insertions: [list of barcodes at site]
                        no_match                list of barcodes with no match to ta site

    """

    tasite_barcode_dict = {}
    no_match        = []

    for line in barcode_list:
        site = line[0]
        strand = line[1]
        barcode = line[2]
        #check if site is in list of ta sites  
        if site in ta_sites:
            #check tasite_barcode_dict for site
            if site in tasite_barcode_dict:
                #get list of barcodes at site
                bc_list = tasite_barcode_dict[site]
                #add barcode to existing list of barcodes
                bc_list.append(barcode)
                #update dictionary
                tasite_barcode_dict[site]=bc_list
            else:
                tasite_barcode_dict[site] = [barcode]
        else:
            # if no ta site matched, look for ta sites up  to 2 nt away to account for matching gtta or tta
            closest_ta = tn.take_closest(ta_sites, site)
            if abs(closest_ta - site) < 3:
                #is closest_ta in dict
                if closest_ta in tasite_barcode_dict:
                    bc_list = tasite_barcode_dict[closest_ta]
                    bc_list.append(barcode)
                    tasite_barcode_dict[closest_ta] = bc_list
                else:
                    tasite_barcode_dict[closest_ta] = [barcode]
            else:
                no_match_str = site, strand, barcode
                no_match.append(no_match_str)

    return tasite_barcode_dict, no_match


In [61]:
sam = "tests/mapped_test_5000_R1.sam"
bc_list = filter_mapped_reads_tag(sam)
bovis_fasta = "ref_seqs/Mbovis_AF2122-97.fasta"
bovis_seq   = tn.open_fasta(bovis_fasta)
ta_sites = tn.find_insertion_sites(bovis_seq)[0]
bc_dict = assign_sites(ta_sites, bc_list)
#for k,v in bc_dict[0].items()#:
#    print(k,v)
print(len(bc_dict[0]))
print(len(bc_dict[1]))
no_match = bc_dict[1]
count = 1
for line in no_match:
    if line[1] == 'F':
        count += 1
        print(line)
print(count)

    
    

Total number of mapped reads:  1180
number of reads with tag:  1173
number of reads with no tag:  7
1001
45
(500641, 'F', 'AGACCTTT')
(812555, 'F', 'AAGACGTC')
(824174, 'F', 'CAGTGGTG')
(1357296, 'F', 'ACCGACGG')
(1775261, 'F', 'AACTCCAA')
(2883723, 'F', 'CGCTAAGG')
(2883723, 'F', 'CTACGGAC')
(2883723, 'F', 'ATTTCCTA')
(2883723, 'F', 'CCAACCTA')
(2883723, 'F', 'CACTTAGG')
(2883723, 'F', 'AAGACCAA')
(2916736, 'F', 'GACTTCGT')
(3719320, 'F', 'TGCATTAA')
(3828970, 'F', 'CTAGCACT')
(3828970, 'F', 'CTAGCACT')
(3926596, 'F', 'CTTCGTGG')
17


Check these no match sites on Artemis. is threshold of 2 nt reasonable?
These sites are stil being calculated wrong:
179561, R  -28 179533
336343, R -31 should be 336312  
373583, F should be 373585  -2 but matched -gtta so this is ok (if threshold <3 will be included)
429332, R, should be 429331 -1 (matched -gtta)
431924, R, should be 431921 -3
435857, R should be 435790, -33 nowhere near read start (or end)
439737, F should be at 439739 -2
500641, F should be 500645 + matches "CTGTTA"
566691, R should be 566641 -50


30 reads in R direction didn't match a TA site. maybe have extra step to catch these? seems like the transposon wasn't soft-clipped? Look at some of these reads


17 forward reads mostly didn't match at all--some from same position:
2883723 F: these reads don't seem to map to this coordinate though this is where they are aligned? no TA site in vicinity 6 reads show on artemis

![forward possible mismapped](images/forward_mismapped.png)

2916736 also mis-mapped (no TA site)
3719320 and this one
500641 not a TA site
812555 not a TA site
3828970 this one looks like should be a TA site but read doesn't seem to match genome here (nor does reverse read)

![poss mismap](images/f_and_r_mismap.png)



Maybe some of the R reads are still including soft-clipping in the alignment start. seems about the length of the transposon. Other reads have sequences that match the end of the transposon, so they are matched earlier than the TA site. I assume the insertion still occurs at TA site, so need to allow some flexibility for these reads.



In [6]:

import scripts.tnseq_pro as tn
samfile = "tests/mapped_test_5000_R1.sam"
data    = tn.read_samfile(samfile)
header  = data[0]
reads   = data[1]

def find_reads(samfile, value):
    for read in reads:
        #print(read.split()[3])
        if int(read.split()[3]) > value -200 and int(read.split()[3]) < value + 200:
            print(read)

find_reads(samfile, 566691)

A01968:63:H77VYDSX5:4:1101:16857:1438_1:N:0:AACGTGAT_BC:CAAAACTA	16	NC_002945.3	566575	60	51S67M33S	*	0	0	GTGGTGTATTCGTAGCTGGATTATCCAGCCTATGGCCGGCCCGGCAAAACCGCGACTCCGTCGATGACGTGCAGCAAAGGAGACATGTAGTGACCGGATCAGCTGGGCCTGACATCTAACAGGTTGGCTGATAAGTCCCCGGTCTCTAGAC	,:,:,,F,:,F,,F,,,,,,,,,,:,,F:,,FF,F,F,,,,,,,,,,FF::,,F,,F::,FFFFFF,FFFFF,FFFFFFFFFFF,FFFFFFFF:FFF:F::F:,F,,:,,F,FFFFFFFF:FFF:FFFFFFFF,F::,FF:FF,FFFFFFF	NM:i:0	MD:Z:67	AS:i:67	XS:i:0


Read left most position is 336227. TA starts at 336312. start of gDNA insert is other end of read (right-most position) which is left position + len of alignment (336227 + 86 = 336313) but should also correspond to tag position. However, maybe because tag is present at both ends--soft clipping at both ends, the prediction is 336343 which is pretty close  to adding TA site 336312 + 32 sc bases from left end of read? 



179561 also two soft clips, 5' soft clip is maybe rc tranposon sequence but too many mismatches?


435857 in read   68S34M33S maps to TA at 435790 which is length of first SC
This one has adapter P7 sequence still attached


566575	predicted to be 566691, 51S67M33S, TA starts at 566641 which is prediction - 51 soft-clipping +1; or left most - align length + 1

GTGGTGTATTCGTAGCTGGATTATCCAGCCTATGGCCGGCCCGGCAAAACC this is 51 clipped from 3' end of read. 

If there are two soft clips, could first subtract first one from prediction before calculating start.

Or if we go back to finding transposon tag in reverse complement of read


Try the complement of read instead of reverse complement--we are searching reverse strand reads backwards, 3' to 5' -- won't solve problem when soft clipping at both ends

Create small set of reads with soft-clipping on both ends to see how the tranposon tag position is identified. 

In [13]:
both_ends_reverse = []
both_ends_forward = []
for read in reads:
    cig = read.split()[5]
    cig_list = tn.parse_cigar(cig)
    strand = tn.parse_samflag(read)
    if len(cig_list) >2 and strand=="R":
        both_ends_reverse.append(read)
        print(cig_list)
    elif len(cig_list) >2 and strand=="F":
        both_ends_forward.append(read)

for read in both_ends_forward:
    tag_pos = tn.find_tags3(read, "ACTTATCAGCCAACCTGTTA", 2)
    ta_pos =  tn.find_ta_position(read, tag_pos)
    print(read, "\n", tag_pos, ta_pos)
    

['29S', '89M', '33S']
['32S', '86M', '33S']
['2S', '116M', '33S']
['2S', '116M', '33S']
['4S', '92M', '33S']
['68S', '34M', '33S']
['51S', '67M', '33S']
['13S', '105M', '33S']
['33S', '85M', '33S']
['2S', '102M', '32S']
['16S', '102M', '33S']
['3S', '115M', '33S']
['5S', '113M', '33S']
['69S', '49M', '33S']
['4S', '114M', '33S']
['4S', '114M', '33S']
['4S', '114M', '33S']
['34S', '84M', '33S']
['25S', '93M', '33S']
['78M', '1D', '40M', '33S']
['78S', '40M', '33S']
['35S', '83M', '33S']
['3S', '97M', '33S']
['19S', '105M', '27S']
['21S', '103M', '27S']
['3S', '115M', '33S']
['19S', '99M', '33S']
['51S', '67M', '33S']
['5S', '113M', '33S']
['65S', '54M', '32S']
['20S', '100M', '31S']
['31S', '87M', '33S']
['52S', '66M', '33S']
['7S', '59M', '33S']
['64S', '54M', '33S']
['43S', '75M', '33S']
A01968:63:H77VYDSX5:4:1101:15347:2644_1:N:0:AACGTGAT_BC:CAGTGTAC	0	NC_002945.3	57868	60	32S79M3S	*	0	0	GTCTAAGACCGGGGACTTATCAGCCAACCTGTTATGACCCGCAGCCACCGGTGGATGCCGGGTCCCTGGCCAAGGCCTCGCCGGCTACCCATCGGCG

### solution to mis-assigning tag position for reverse reads with soft-clipping at both ends of alignment

Are the added bases on 3' end read-through of adapters that weren't removed from a shorter-than-average gDNA fragment?

"for reads coming from fragments shorter than the read length, the adapter might appear at the other end of R1, TACCACGACCA. If so, the adapter suffix is stripped off." from tpp docs

The find_tag program is finding tag from beginning of entire read--including soft-clipped bases at the 3' end of the reverse reads (left most end of sequence searched). Reduce search space and get more accurate position by having mmfind begin at the start of the aligned sequence (slicing with seq[soft-clipped base position: len(sequence]). Using rc of sequence just shifts problem to other end which also has soft-clipped bases that may be longer than tag length (so need to add to tag position) and also requires more time to find rc of each read vs using rc of tag only.

In [ ]:
def parse_cigar(cigar):
    """
    Parse cigar string from sam file
    regex: \*|([0-9]+[MIDNSHPX=])+
    """
    import re
    #find all number/letter combos in cigar string
    cig_list = re.findall(r'[0-9]+[MIDNSHPX=]+', cigar)
    return cig_list

In [45]:
def mmfind1(G,n,H,m,max): # lengths; assume n>m
  """
  A function to find matches of transposon sequence in read 
  (from tpp_tools.py, https://github.com/mad-lab/transit/blob/master/src/pytpp/tpp_tools.py)
    Input               G                           sequence string
                        n                           length of read sequence
                        H                           pattern string
                        m                           length of pattern
                        max                         maximum mismatches
    Output              i, -1                       start position of match, or -1 for no match
  """

  a = G[:n].find(H[:m])
  if a!=-1: return a # shortcut for perfect matches
  for i in range(0,n-m):  # range of 0 to difference between seq length and len pattern
    cnt = 0
    for k in range(m):
      if G[i+k]!=H[k]: cnt += 1
      if cnt>max: break
    if cnt<=max: return i
  return -1


def find_tags3(read, target_tag, max):
    """
    Find transposon tag in sequence of each read.

      Input           read              mapped read
                      target_tag        string that matches transposon sequence
                      max               maximum number of mismatches allowed (default=2)
      Output          start             calculation of start of read/insertion point 
                                        from left-most start position of tag or -1 if no match
    """
    from Bio.Seq import Seq
    #find rc of tag
    tag_seq = Seq(target_tag)
    rc_tag  = str(tag_seq.reverse_complement())
    # find strand of read
    strand = parse_samflag(read)
    
    seq = read.split()[9]
    if strand == "F":
        #search string for transposon seq with max num mismatches
        match = mmfind1(seq, len(seq), target_tag, len(target_tag), max)
        if match != -1:
            start = match + len(target_tag) #this gives start position of read (start of ta site)
        else:
            start = match
    else: ##strand is reverse start searching sequence from other end 
        #search reverse sequence (3' to 5') for rc of tag (starting with TAAC--) start position of tag is end of tag and start of insert/ta site
        #in some cases, soft-clipped bases at 3' end of read will make tag too long
        # parse cigar string for relevant soft-clipping at 3' end of read
        cig = read.split()[5]
        cig_list = parse_cigar(cig)
        if cig_list[0][-1]=="S": #if there is soft-clipping at both ends of read
            sc_correction = int(cig_list[0][:-1]) #number of soft-clipped bases
            match = mmfind1(seq[sc_correction:len(seq)], len(seq), rc_tag, len(rc_tag), max)
        else:
            match = mmfind1(seq, len(seq), rc_tag, len(rc_tag), max)
        start = match
    
    return start


In [18]:

bovis_fasta = "ref_seqs/Mbovis_AF2122-97.fasta"
bovis_seq   = tn.open_fasta(bovis_fasta)
ta_sites = tn.find_insertion_sites(bovis_seq)[0]
count = 0
for read in both_ends_reverse:
    tag_pos = tn.find_tags3(read, "ACTTATCAGCCAACCTGTTA", 2)
    ta_pos =  tn.find_ta_position(read, tag_pos)
   # print(read, "\n", tag_pos, ta_pos)

    if int(ta_pos) in ta_sites:
        count +=1
    #else:    
       # print(read, "\n", tag_pos, ta_pos)
print(len(both_ends_reverse))  
print("number of correct matches on reverse strand: ", count)

count=0
for read in both_ends_forward:
    tag_pos = tn.find_tags3(read, "ACTTATCAGCCAACCTGTTA", 2)
    ta_pos =  tn.find_ta_position(read, tag_pos)
   # print(read, "\n", tag_pos, ta_pos)

    if int(ta_pos) in ta_sites:
        count +=1
    else:    
        print(read, "\n", tag_pos, ta_pos)
print(len(both_ends_forward))   
print("number of correct matches on forward strand: ", count)

36
number of correct matches on reverse strand:  35
A01968:63:H77VYDSX5:4:1101:13386:2472_1:N:0:AACGTGAT_BC:TAGGCCGT	0	NC_002945.3	134437	60	32S72M47S	*	0	0	GTCTAGAGACCCGGGACTTATCAGCCAACCTGTTACTTCACCCAGGGCGCTCCACCCTCGCCCCTACAGCACACCTGGTCGTTGGGGGGGGGGGATCAGTATTAAGTTCTATGGCCCCAGTTGCATATACGGGCGACCCTACAGTTGGCGG	FFFF:::FFFF,F:F:,FFF::FFFF:F,FF,F::FF:FFFF:F,,FFFF,FFFFF,F:FF:F,,FFFFF:FF:::FFF::F,,,F,F,:F,F,F,FFF,,,FF,,,F,F,,,,F,,,:F,:,F,,:F,,:F,F:FF,::FF,,,F::FFF	NM:i:3	MD:Z:56T2A3G8	AS:i:57	XS:i:0 
 35 134437
A01968:63:H77VYDSX5:4:1101:4607:2425_1:N:0:AACGTGAT_BC:ACGCTCCC	0	NC_002945.3	925690	60	32S114M5S	*	0	0	GTCTAGAGACCGGGGACTTATCAGCCAACCTGTTACACTGCTTCCTGCTGATTCCTCCCTGCGATCGGTCGATCGCAGGATCGGTTGGCATCGAGGTCATGTCGCTGTGGGAGGAGATGTCGCGTGTCTTATGTGAGCGTGTTGCCAGATC	FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF:FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF:F:FFF:FFFFFFFFFFFF:FFFFFFFFFFFFFFFFFFFFFFFFFF	NM:i:0	MD:Z:114	AS:i:114	XS:i:0 
 35 925690
A01968:63:H77VYDSX5:4:1101:19506:1047_1

The read that doesn't work ends in TA but looks really badly mapped to genome. Why is this? There were quite a few of these cases in the forward reads, as well. Could be that these have phage or contaminant DNA which won't map to our genome.

Blast some of the misaligned read sequences (without the tags at either end)

In [9]:
coords = [3828970, 2883723, 2916736, 500641, 3719320]

#find reads with left-most coordinate at these sites
import scripts.tnseq_pro as tn

reads = tn.read_samfile("tests/mapped_test_5000_R1.sam")[1]
for read in reads:
    left_coord = int(read.split()[3])
    cigar = read.split()[5]
    cig_string = tn.parse_cigar(cigar)
    strand = tn.parse_samflag(read)
    seq = read.split()[9]
    if left_coord in coords:
        print(read)
        if cig_string[0][-1]=="S":
            sc = int(cig_string[0][:-1])
            align_len = int(cig_string[1][:-1])
            align_seq = seq[sc:sc+align_len]
            print(align_seq)

A01968:63:H77VYDSX5:4:1101:32904:2519_1:N:0:AACGTGAT_BC:AGACCTTT	0	NC_002945.3	500641	60	29S122M	*	0	0	GTCTAGAGACCGGGGACTTATCAGCCAACCTGTTAGGACCTCATCTTTCGGCTAACGCAACGAAACTCTATAAGCGAATTCTAAGAGAAGGTTCCGACAGATGGTGTTAGGCATACGCAATTGCCCAGTTGCCAGCCTGCATATTCTGAAC	FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF:FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF	NM:i:1	MD:Z:104C17	AS:i:117	XS:i:0
CTGTTAGGACCTCATCTTTCGGCTAACGCAACGAAACTCTATAAGCGAATTCTAAGAGAAGGTTCCGACAGATGGTGTTAGGCATACGCAATTGCCCAGTTGCCAGCCTGCATATTCTGAAC
A01968:63:H77VYDSX5:4:1101:19506:1047_1:N:0:AACGTGAT_BC:CGCTAAGG	0	NC_002945.3	2883723	60	40S51M43S	*	0	0	GTCTAGAGACCGGGGACTTATCAGCCAACCTGTTACTAGACCGTCCAGTCTGGCAGGCCGGAAACATCGGTCAGCAGATAGGCTTTACCAGTAAGAAGGAGATATACATATGGAAAAAAAGGAATTACGTTTTA	FFFFFFFFFFFFFF:FFFFFFFFFF:FFF:FFFFFFFF:FFF,F::F:FFFFFFFFFF,FF:FFFFFF:,FFFFFFFFFFF:FFFFFF,FF:FFFF:FFFFFFFFF:F,FFF:FFF,FF,,:FF:F,F,F,FFF	NM:i:0	MD:Z:51	AS:i:51	XS:i:0
CCGTCCAGTCTGGCAGGCCGGAAACATCG

all of the misaligned reads are from mbovis genome, so used artemis to find where they should be aligning

500641  has -CTGTTA at start of sequence, so not aligning with TA site, but is correctly mapped

2883723 should be 2888134 (-4411)  (6 reads)
2916736 should be at 2921152  (-4416)
3719320 should be 3723733-816 (-4413)
3828970 should be 3833383-3497  (-4413)  (2 reads)

Why have these reads mapped incorrectly? Is it because of tag? correctly soft-clipped and have correct length of aligned sequence, just wrong left-most coordinate. why the consistent distance between where they should be starting and where they are reported to start?

3719320 doesn't map to TA but there is a TA downstream (further along), restrict to TA sites upstream (earlier) that were missed by mapping matching end of transposon tag along with insert

In [44]:
# how many forward reads not mapping to ta site
bovis_fasta = "ref_seqs/Mbovis_AF2122-97.fasta"
bovis_seq   = tn.open_fasta(bovis_fasta)
ta_sites = tn.find_insertion_sites(bovis_seq)[0]
tn_tag = "ACTTATCAGCCAACCTGTTA"
reads = tn.read_samfile("tests/mapped_test_5000_R1.sam")[1]
fwd_count = 0
match_count = 0
for read in reads:
    if tn.parse_samflag(read) == "F":
        fwd_count +=1
        tag_pos = tn.find_tags3(read, tn_tag, 2)
        ta_pos =  tn.find_ta_position(read, tag_pos)
        #print(read, "\n", tag_pos, ta_pos)
        if ta_pos in ta_sites:
            match_count +=1
        else:
            # if no ta site matched, look for ta sites up  to 4 bases away to account for matching gtta or tta
            if ta_pos != "-":
                closest_ta = tn.take_closest(ta_sites, int(ta_pos))
                if ta_pos - closest_ta < 4:
                    match_count +=1
                else: 
                    print(read, "\n", tag_pos, ta_pos)
print("total number of reads: ", fwd_count)   
print("number of correct matches on forward strand: ", match_count)
print("percent of no matches on forward strand: ", round(((fwd_count-match_count)/fwd_count)*100, 1), "%")

A01968:63:H77VYDSX5:4:1101:19913:1783_1:N:0:AACGTGAT_BC:AAGACGTC	0	NC_002945.3	812555	60	40S42M	*	0	0	GTATAGATACCGGGGACTTATCAGCCAACCTGTTACTCCTCGATGTTGTTCGCTTTCGTGCCGCTGCGCACTATTCTCCCGC	,:,FF:,,F:FF:,,FFF:F,:F:,:FFFFFF,FFF,F:,F,FF,F:FFFF:FF,,F,F,::FF:F,,FFF,FF,,F:,F,,	NM:i:2	MD:Z:14G26G0	AS:i:36	XS:i:0 
 35 812555
A01968:63:H77VYDSX5:4:1101:18403:2926_1:N:0:AACGTGAT_BC:ACCGACGG	0	NC_002945.3	1357296	60	37S70M	*	0	0	GTCTAGAGACCGGGGACTTATCAGCCAACCTGTTATATCGCAGAACTGGCGGTTGTCTGGTCTGGCGGGTGAGTACATCACCCCGGTGCCGGCACAGCAGCGCCTCA	:FFFF,FFF:,:FFFFFFFF:::F:F,FF:FF:FFF,FFFFFFFF:F,FFFFFFF:F,:FF,FFF,,,FFFFFFFFFFFF,FFFFFFFFFFFFFFF,F:FF:FF,FF	NM:i:1	MD:Z:69G0	AS:i:69	XS:i:0 
 35 1357296
A01968:63:H77VYDSX5:4:1101:19506:1047_1:N:0:AACGTGAT_BC:CGCTAAGG	0	NC_002945.3	2883723	60	40S51M43S	*	0	0	GTCTAGAGACCGGGGACTTATCAGCCAACCTGTTACTAGACCGTCCAGTCTGGCAGGCCGGAAACATCGGTCAGCAGATAGGCTTTACCAGTAAGAAGGAGATATACATATGGAAAAAAAGGAATTACGTTTTA	FFFFFFFFFFFFFF:FFFFFFFFFF:FFF:FFFFFFFF:FFF,F::F:FFFFFFFFFF,FF:FFFFFF:,FFFFFFFFFF

Nothing to do with soft-clipping on 3' end of forward read (most that miss a ta site don't have that anyway).


Raising threshold of distance to TA site to <4 eliminates those with matching bases before and including -GTTA. Remaining unmatched are result of mismatching described above. Will address this separately.

Not ideal to make such lenient TA matching as some TA sites are very close together. But all of these reads are positive for containing the tag--not sure it matters much? it is possible (but unlikely) that there is a case where read is assigned to an upstream TA site within 3nt of true TA site, will with high probability be within the same gene. This threshold seems to leave only reads where the mapping isn't great and there isn't a TA site at the start of the read.

But when assign_counts_to_sites implemented, not differently applied by strand (strand info lost in insertion list) so still need to use absolute value of difference as for reverse strand, predicted insertion will be 2-3 bases further downstream than ta site. Could process each strand separately for assigning counts to ta site, and then combine together when creating dictionary and wig file.

    Would need two separate scripts for assign_counts_to_sites: 
        for forward strand:  if (ta_pos - closest_ta) < 4  (actual TA is further downstream--higher coordinate)
        for reverse strand:  if (closest - ta_pos) < 4     (actual TA is further upstream of predicted--lower coordinate)


First divide reads by strand, then filter reads by tag and determine TA site position. Then assign to TA site.

In [3]:
def filter_reads_by_strand(reads):
    """
    Function to filter reads by strand
    Input               reads           list of reads
    Output              fwd_strand_reads    list of reads mapped to forward strand
                        rev_strand_reads    list of reads mapped to reverse strand
    """
    fwd_strand_reads = []
    rev_strand_reads = []
    for read in reads:
        if tn.parse_samflag(read) == "F":
            fwd_strand_reads.append(read)
        else:
            rev_strand_reads.append(read)
    return fwd_strand_reads, rev_strand_reads

In [4]:
def find_tags_fwd(read, target_tag, max):
    """
    Find transposon tag in sequence of each forward read.

      Input           read              mapped read to forward strand (+)
                      target_tag        string that matches transposon sequence
                      max               maximum number of mismatches allowed (default=2)
      Output          start             calculation of start of read/insertion point 
                                        from left-most start position of tag or -1 if no match
    """
    
    seq = read.split()[9]
    
    #search string for transposon seq with max num mismatches
    match = mmfind1(seq, len(seq), target_tag, len(target_tag), max)
    if match != -1:
        start = match + len(target_tag) #this gives start position of read (start of ta site)
    else:
        start = match
    
    return start



In [5]:
def find_tags_rev(read, target_tag, max):

    from Bio.Seq import Seq
    #find rc of tag
    tag_seq = Seq(target_tag)
    rc_tag  = str(tag_seq.reverse_complement())
    seq = read.split()[9]
    ##strand is reverse start searching sequence from other end 
    #search reverse sequence (3' to 5') for rc of tag (starting with TAAC--) start position of tag is end of tag and start of insert/ta site
    #in some cases, soft-clipped bases at 3' end of read will make tag too long
    cig = read.split()[5]
    cig_list = parse_cigar(cig)
    if cig_list[0][-1]=="S": #if there is soft-clipping at 3' end of reverse read
        sc_correction = int(cig_list[0][:-1]) #number of soft-clipped bases
        match = mmfind1(seq[sc_correction:len(seq)], len(seq), rc_tag, len(rc_tag), max) #start search from start of aligned bases
    else:
        match = mmfind1(seq, len(seq), rc_tag, len(rc_tag), max)
    
    return match

In [8]:
import scripts.tnseq_pro as tn

def filter_mapped_reads_tag3(sam_file, tag="ACTTATCAGCCAACCTGTTA", mismatch_max=2):
    """ 
    Revised filtering function: filters reads for presence of transposon tag and for duplicate barcode/ta-position

    Input               sam_file                            .sam file of mapped reads
                        tag                                 str of transposon tag used, default is Himar1
                        mismatch_max                        integer of maximum mismatches allowed in search for tag
    Output              read barcodes                       list of 3 values: insert_start, strand, barcode from reads with insertions
                                                        
    """
    import sys
    import os

    #check tag has valid nucleotides
    if tn.check_seq(tag):
        pass
    else:
        sys.exit("Invalid sequence tag")

    #read sam_file and sort lines between header and reads
    data    = tn.read_samfile(sam_file)
    header  = data[0]
    reads   = data[1]

    #divide reads by strand
    fwd_strand_reads, rev_strand_reads = tn.filter_reads_by_strand(reads)
    
    tagged_reads    = []
    barcode_list_fwd    = []
    barcode_list_rev    = []
    barcode_list_all    = []
    notag_count = 0

    for read in fwd_strand_reads:
        tag_pos   = tn.find_tags_fwd(read, tag, mismatch_max)
        strand    = "F"
        if tag_pos != -1:  #there is match for tranposon tag
            insertion_coord = tn.find_ta_position(read, tag_pos)
            #find barcode
            read_name  = read.split()[0]
            barcode    = read_name.split("BC:",1)[1]
            pos_barcode = [insertion_coord, strand, barcode]
            #add to list of all reads with insertions
            barcode_list_fwd.append(pos_barcode) 
            #append read to tagged reads
            tagged_reads.append(read)
        else:
            notag_count += 1
  
    print("Total number of mapped reads forward strand: ", len(fwd_strand_reads))
    print("number of forward reads with tag: ", len(barcode_list_fwd))
    print("number of forward reads with no tag: ", notag_count)
  
    
    notag_count = 0

    for read in rev_strand_reads:
        tag_pos   = tn.find_tags_rev(read, tag, mismatch_max)
        strand    = "R"
        if tag_pos != -1:  #there is match for tranposon tag
            insertion_coord = tn.find_ta_position(read, tag_pos)
            #find barcode
            read_name  = read.split()[0]
            barcode    = read_name.split("BC:",1)[1]
            pos_barcode = [insertion_coord, strand, barcode]
            #add to list of all reads with insertions
            barcode_list_rev.append(pos_barcode) 
            #append read to tagged reads
            tagged_reads.append(read)
        else:
            notag_count += 1
  
    print("Total number of mapped reads reverse strand: ", len(rev_strand_reads))
    print("number of forward reads with tag: ", len(barcode_list_rev))
    print("number of forward reads with no tag: ", notag_count)

    #write read files (includes all fwd and reverse reads)
    bn = os.path.basename(sam_file)
    outfile = "tag_filtered_templates_" + bn
    with open(outfile, 'w') as f:
        for line in header:
            f.write(f"{line}\n")
        for line in tagged_reads:
            f.write(f"{line}\n")
  
    barcode_list_all = barcode_list_fwd + barcode_list_rev
    return barcode_list_all

In [10]:
import scripts.tnseq_pro as tn
mySam = "tests/mapped_test_5000_R1.sam"
bc_lists = filter_mapped_reads_tag3(mySam, tag="ACTTATCAGCCAACCTGTTA", mismatch_max=2)

Total number of mapped reads forward strand:  581
number of forward reads with tag:  580
number of forward reads with no tag:  1
Total number of mapped reads reverse strand:  599
number of forward reads with tag:  593
number of forward reads with no tag:  6


In [8]:
import scripts.tnseq_pro as tn
mySam = "tests/mapped_test_5000_R1.sam"
bc_lists = tn.filter_mapped_reads_tag3(mySam, tag="ACTTATCAGCCAACCTGTTA", mismatch_max=2)
#assign to ta sites and make dictionary
bovis_fasta = "ref_seqs/Mbovis_AF2122-97.fasta"
bovis_seq   = tn.open_fasta(bovis_fasta)
ta_sites = tn.find_insertion_sites(bovis_seq)[0]
results = tn.assign_counts_to_sites(ta_sites, bc_lists)
ta_dict = results[0]
no_match = results[1]
print(len(no_match))
print(len(ta_dict))


Total number of mapped reads forward strand:  581
number of forward reads with tag:  580
number of forward reads with no tag:  1
Total number of mapped reads reverse strand:  599
number of forward reads with tag:  593
number of forward reads with no tag:  6
8
1032


In [7]:
def sam_to_wig(samfile, genome_fasta, sample_name):
    """
    Wrapper function to create wig file from sam file
    Input           samfile         sam file of mapped reads   
                    genome_fasta    path to fasta file of genome        str
                    sample_name     name of sample                      str
    Output          wig file        wig file of insertions
    """
    import os
    fasta       = genome_fasta
    #get genome name from fasta file
    genome      = os.path.basename(fasta).split(".")[0]
    fasta_seq   = open_fasta(genome_fasta)
    ta_sites    = find_insertion_sites(fasta_seq)[0]
    # make list of the positions of every insertion from unique templates
    read_positions = filter_mapped_reads_tag3(samfile, tag="ACTTATCAGCCAACCTGTTA", mismatch_max=2)
    #reduce read positions and barcodes to unique templates
    template_positions = reduce_to_templates(read_positions)

    #count number of reads per ta site
    res=assign_counts_to_sites(ta_sites, template_positions)
    #make complete dictionary of all possible ta sites including those with no insertions
    ta_dict = ta_sites_to_dict(ta_sites, res[0])
    #write wig file
    wig_file = write_wig_from_dict(ta_dict, sample_name, genome)
    
    return wig_file

In [1]:
import scripts.tnseq_pro as tn
mySam = "tests/mapped_test_5000_R1.sam"
tn.sam_to_wig(mySam, "ref_seqs/Mbovis_AF2122-97.fasta", "test")

Total number of mapped reads forward strand:  581
number of forward reads with tag:  580
number of forward reads with no tag:  1
Total number of mapped reads reverse strand:  599
number of forward reads with tag:  593
number of forward reads with no tag:  6


'output/test_insertions.wig'

## need to add function to reduce starts/barcodes to unique templates. 

can do before assigning ta sites as duplicates will have same starts. between filter_mapped_tags and 

In [24]:
def remove_dups(fwd_bc_list, rev_bc_list):
    """
    Function to remove duplicate barcodes from fwd and rev lists
    Input               fwd_bc_list         list of fwd barcodes
                        rev_bc_list         list of rev barcodes
    Output              template_bc_list    list of unique barcodes from fwd and rev lists
    
    """
    import numpy as np

    fwd_bc_arr = np.unique(np.array(fwd_bc_list), axis=0)
    rev_bc_arr = np.unique(np.array(rev_bc_list), axis=0)
    fwd_bc_list = fwd_bc_arr.tolist()
    rev_bc_list = rev_bc_arr.tolist()
    template_bc_list = fwd_bc_list + rev_bc_list
    
    return template_bc_list

In [ ]:
import scripts.tnseq_pro as tn
mySam = "tests/mapped_test_5000_R1.sam"
import os
fasta       = "ref_seqs/Mbovis_AF2122-97.fasta"
fasta_seq   = tn.open_fasta(fasta)
ta_sites    = tn.find_insertion_sites(fasta_seq)[0]
# make list of the positions of every insertion from unique templates
read_positions = tn.filter_mapped_reads_tag3(mySam, tag="ACTTATCAGCCAACCTGTTA", mismatch_max=2)
#reduce read positions and barcodes to unique templates
template_positions = remove_dups(read_positions[0], read_positions[1])
print("Number of unique template reads: ", len(template_positions))

In [3]:
def sam_to_wig(samfile, genome_fasta, sample_name):
    """
    Wrapper function to create wig file from sam file
    Input           samfile         sam file of mapped reads   
                    genome_fasta    path to fasta file of genome        str
                    sample_name     name of sample                      str
    Output          wig file        wig file of insertions
    """
    import os
    
    #get genome name from fasta file
    genome      = os.path.basename(genome_fasta).split(".")[0]
    fasta_seq   = tn.open_fasta(genome_fasta)
    ta_sites    = tn.find_insertion_sites(fasta_seq)
    # make list of the positions of every insertion
    read_positions = tn.filter_mapped_reads_tag3(samfile, tag="ACTTATCAGCCAACCTGTTA", mismatch_max=2)
    #reduce read positions and barcodes to unique templates
    template_positions = tn.remove_dups(read_positions[0], read_positions[1])
    #count number of reads per ta site
    res=tn.assign_counts_to_sites(ta_sites, template_positions)
    #make complete dictionary of all possible ta sites including those with no insertions
    ta_dict = tn.ta_sites_to_dict(ta_sites, res)
    #write wig file
    wig_file = tn.write_wig_from_dict(ta_dict, sample_name, genome)
    
    return wig_file

In [2]:
import scripts.tnseq_pro as tn
mySam = "tests/mapped_test_5000_R1.sam"
fasta_seq = tn.open_fasta("ref_seqs/Mbovis_AF2122-97.fasta")
ta_sites = tn.find_insertion_sites(fasta_seq)
#print(ta_sites[0:10])
positions = tn.filter_mapped_reads_tag3(mySam, tag="ACTTATCAGCCAACCTGTTA", mismatch_max=2)
print(positions[0][0:10])
print(positions[1][0:10])
temp_pos = tn.remove_dups(positions[0], positions[1])
print(temp_pos[0][0:10])
print(temp_pos[1][0:10])
res = tn.assign_counts_to_sites(ta_sites, temp_pos[0], temp_pos[1])


Total number of mapped reads forward strand:  581
number of forward reads with tag:  580
Total number of mapped reads reverse strand:  599
number of forward reads with tag:  593
[[26121, 'F', 'CGCCACCT'], [31581, 'F', 'CCGACTGA'], [37337, 'F', 'AATCACTC'], [39857, 'F', 'CGAGACAG'], [47375, 'F', 'ACAGGATC'], [51951, 'F', 'GACCAAGC'], [57868, 'F', 'CAGTGTAC'], [87311, 'F', 'TGTTCCAG'], [87640, 'F', 'AACCAGAA'], [99975, 'F', 'AAGCGCTT']]
[[20348, 'R', 'GAACAACT'], [35234, 'R', 'TATTCTAA'], [39857, 'R', 'TGAGGCGA'], [48092, 'R', 'CTGCTCCA'], [52821, 'R', 'CATAGAAT'], [63367, 'R', 'TTACTATT'], [69579, 'R', 'TTTAGTCA'], [72386, 'R', 'TTACCATA'], [72386, 'R', 'TTACCATA'], [81201, 'R', 'ATATTCCT']]
[['1004830', 'F', 'AGTAGCCC'], ['1019979', 'F', 'GAATAAAT'], ['1031012', 'F', 'TACTAATT'], ['1041105', 'F', 'TGACATCT'], ['1043320', 'F', 'AATTGCAT'], ['1057494', 'F', 'TCATTCAT'], ['1066421', 'F', 'GACACGAG'], ['1071009', 'F', 'ACCCTCCT'], ['108988', 'F', 'TACTTTTC'], ['1111361', 'F', 'GCCGCCCT']]


In [1]:
import scripts.tnseq_pro as tn
mySam = "tests/mapped_test_5000_R1.sam"
tn.sam_to_wig(mySam, "ref_seqs/Mbovis_AF2122-97.fasta", "test")

Total number of mapped reads forward strand:  581
number of forward reads with tag:  580
Total number of mapped reads reverse strand:  599
number of forward reads with tag:  593


'output/test_insertions.wig'

## re-test with whole dataset

In [2]:
#iterate through samples and run sam_to_wig script

# iterate through files
import scripts.tnseq_pro as tn

# make list of .sam files in directory
import os
import glob
import re
sam_files = glob.glob("sorted_reads" + "/*.sam")
print(sam_files)
bovis_fasta = "ref_seqs/Mbovis_AF2122-97.fasta"
for file in sam_files:
    #find sample name from file
    sample_filename = os.path.basename(file).split(".")[0]
    sample_name = re.findall(r'mapped_(\w*)_R1_001', sample_filename)[0]
    print(sample_name)
    print(file)
    tn.sam_to_wig(file, bovis_fasta, sample_name)

['sorted_reads/mapped_A1_R1_001.sam']
A1
sorted_reads/mapped_A1_R1_001.sam


iterrupted kernel after 75 minutes. finding tag position using mmfind seems to be the hold-up. Could try on server to see if it is any faster. Not getting any of my counter statements back--not processing even 100000 reads after 30 min.

Need to trim transposon tag first and then map so that the left-most coordinate is accurate and no need to search for tag? 

Or reduce search space for transposon tag. It has to start in the first 20 nt of read (on fwd strand) and in the final 40 nt of read on rev strand (or first 20 of rc of read).

In [4]:
def find_tags_fwd2(read, target_tag, max):
    import scripts.tnseq_pro as tn
    """
    Find transposon tag in sequence of each forward read. Needs to be within first 20 nt of read (start between 5-10)

      Input           read              mapped read to forward strand (+)
                      target_tag        string that matches transposon sequence
                      max               maximum number of mismatches allowed (default=2)
      Output          start             calculation of start of read/insertion point 
                                        from left-most start position of tag or -1 if no match
    """
    
    seq = read.split()[9]
    # for forward strand, search space restricted to first 20 nt of read plus length of tag?
    seq_space = seq[0:20+len(target_tag)]
    #search string for transposon seq with max num mismatches
    match = tn.mmfind1(seq_space, len(seq_space), target_tag, len(target_tag), max)
    if match != -1:
        start = match + len(target_tag) #this gives start position of read (start of ta site)
    else:
        start = match
    
    return start

In [1]:
# could use this to trim transposon tag from read after mapping. then more clear where insertion is
# for forward reads
import scripts.tnseq_pro as tn
mySam = "tests/mapped_test_5000_R1.sam"
reads = tn.read_samfile(mySam)[1]
tag_trimmed_reads = []
fwd_reads = tn.filter_reads_by_strand(reads)[0]
for read in fwd_reads:
    start_pos = tn.find_tags_fwd2(read, "ACTTATCAGCCAACCTGTTA", 2)
    if start_pos != -1:
        new_read = read.replace(read.split()[9], read.split()[9][start_pos:])
        tag_trimmed_reads.append(new_read)
print(len(fwd_reads))
print(len(tag_trimmed_reads))

#for reverse reads
rev_reads = tn.filter_reads_by_strand(reads)[1]
for read in rev_reads:
    start_pos = tn.find_tags_rev(read, "ACTTATCAGCCAACCTGTTA", 2)
    if start_pos != -1:
        new_read = read.replace(read.split()[9], read.split()[9][0:start_pos])
        print(read)
        print(start_pos)
        print(new_read)
        tag_trimmed_reads.append(new_read)
print(len(rev_reads))
print(len(tag_trimmed_reads))


581
580
A01968:63:H77VYDSX5:4:1101:8757:2503_1:N:0:AACGTGAT_BC:GAACAACT	16	NC_002945.3	20232	60	118M33S	*	0	0	ATACGCGTTCGATGACCTCGGTGCCGGCCGCCGTAATCGGCGACTTATTTCGTGGGCGGGTGCGCAGTGGGCGGCGGGCTCCGTGCGAGATGCGTGCCAGGATGGCCAGCAATATGTAACAGGTTGGCTGATAAGTCCCCGGTCTCTAGAC	FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF,FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF	NM:i:0	MD:Z:118	AS:i:118	XS:i:0
116
A01968:63:H77VYDSX5:4:1101:8757:2503_1:N:0:AACGTGAT_BC:GAACAACT	16	NC_002945.3	20232	60	118M33S	*	0	0	ATACGCGTTCGATGACCTCGGTGCCGGCCGCCGTAATCGGCGACTTATTTCGTGGGCGGGTGCGCAGTGGGCGGCGGGCTCCGTGCGAGATGCGTGCCAGGATGGCCAGCAATATG	FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF,FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF	NM:i:0	MD:Z:118	AS:i:118	XS:i:0
A01968:63:H77VYDSX5:4:1101:24089:1595_1:N:0:AACGTGAT_BC:TATTCTAA	16	NC_002945.3	35118	60	118M33S	*	0	0	GTACCGGGCGGCGCTGATGCTCAGCCTAAAAGATGCGATCAGCC

this actually removes the TA bases from the read--will map starting at base after 'TA'. Site list of poss TA sites is start of TA, so 2 bases earlier (upstream) from start position.

fwd: ta site==start position -2, reverse: ta-site==start position +2, but clipping this does nothing for the left most mapped position, just length of read.

search and trim the fastq read--easier because will all be at start of read even if eventually mapped on reverse strand

## find and trim transposon tag from adapter-trimmed, barcoded read 


In [13]:
def find_tags_fastq(seq, target_tag, max):
    
    """
    Find transposon tag in sequence of each forward read. Needs to be within first 20 nt of read (start between 5-10)

      Input           read              mapped read to forward strand (+)
                      target_tag        string that matches transposon sequence
                      max               maximum number of mismatches allowed (default=2)
      Output          start             calculation of start of read/insertion point 
                                        from left-most start position of tag or -1 if no match
    """
    import scripts.tnseq_pro as tn
    
    # for forward strand, search space restricted to first 20 nt of read plus length of tag?
    seq_space = seq[0:20+len(target_tag)]
    #search string for transposon seq with max num mismatches
    match = tn.mmfind1(seq_space, len(seq_space), target_tag, len(target_tag), max)
    
    return match

In [17]:
def trim_tag_fastq(fastq_file, outdir, tag="ACTTATCAGCCAACCTGTTA", mismatch_max=2):
    """
    Function to trim transposon tag from fastq reads
    Input               fastq_file          fastq file of barcoded reads
                        target_tag          string of transposon tag
                        max                 maximum number of mismatches allowed
    Output              tag_trimmed_fasta   fasta file of trimmed reads (header and sequence only)
    """
    import os.path
    import re
    filename = str(fastq_file)
    bn_sample = os.path.basename(fastq_file)
    sample = re.sub(".fastq", "", bn_sample)
    tagged_list = []
    notag_list = []
    with open (fastq_file, 'r') as f:
    # header and sequence for R1 reads
        for index, line in enumerate(f):
            if index % 4 == 0:
                head = line.rstrip()
            # second line and every 4  (sequence)  
            if index %4 == 1:
                seq = line.rstrip()
                #find transposon tag in new sequence
                tag_start = find_tags_fastq(seq, tag, mismatch_max)
                tag_end = tag_start + len(tag)
                if tag_start != -1:
                    new_seq = seq[tag_end:]
                    tagged_list.append(head + "\n")
                    tagged_list.append(new_seq + "\n")
                else:
                    notag_list.append(head + "\n")
                    notag_list.append(seq + "\n")
    # write new file with tagged and no-tagged reads
    new_filename = outdir + "tag_trimmed_" + sample + ".fastq"
    with open(new_filename, 'w') as outfile:
        outfile.writelines(tagged_list)
    outfile.close()
    bad_filename = outdir + "no_tag_" + sample + ".fastq"
    with open(bad_filename, 'w') as outfile:
        outfile.writelines(notag_list)
    
    return new_filename

In [18]:
my_fastq = "tests/barcode_test_5000_R1.fastq"
trim_tag_fastq(my_fastq, "tests/")

'tests/tag_trimmed_barcode_test_5000_R1.fastq'

In [20]:
#try with full barcoded file
my_fastq = "barcoded/barcode_trimmed_A1_R1_001.fastq"
trim_tag_fastq(my_fastq, "barcoded_output/")

'output/tag_trimmed_barcode_trimmed_A1_R1_001.fastq'

This takes 3m 34 s for entire file (~65 million reads)


## Map tag-trimmed fastq with bwa

In [ ]:
# with snakemake (maps, sorts, indexes and creates flagstats report)
#make config.yaml file

cd ~/tn_seq/menadione_tnseq/
conda activate snakemake
snakemake -np -s ~/snakemake/tnseq/mapping/snakefile.smk
snakemake --cores 2 -s ~/snakemake/tnseq/mapping/snakefile.smk
#!snakemake -np -s $my_path/snakemake/tnseq/snakefile.smk
#!nohup snakemake --cores 8 -s $my_path/snakemake/map_bwa/pe/snakefile.smk > nohup_map.out 2>&1 &

takes about 10 minutes to map whole file

## Next step: Matching mapped reads to insertion site (TA-site)

In [ ]:
def find_ta_position(read, tag_pos):
    """
    Find the TA position relative to + strand
    Input           read                        sequencing read (mapped) from sam file
                    tag_pos                     position of tag in read (from find_tags3)
    Output          ta_position                 predicted position of insertion site in read
                                                (end of tag)


    """
    
    strand      = parse_samflag(read)
    left_pos    = read.split()[3]
    
    if strand == 'F':
            ta_position = int(left_pos) - 2
        else: #reverse strand
            ta_position = int(left_pos) + tag_pos
    else:
        ta_position = "-"
    return ta_position

In [31]:
def make_barcode_dict(reads):
    counter = 0
    dup_count = 0
    for read in reads:
        start_pos = read.split()[3]
        counter += 1
        #find barcode
        read_name  = read.split()[0]
        barcode    = read_name.split("BC:",1)[1]
        #add to dictionary of starts and barcodes 
        barcode_dict = {}
        if start_pos in barcode_dict:
            barcode_list = barcode_dict[start_pos]
            if barcode in barcode_list:
                pass
                dup_count += 1
            else:
                barcode_list.append(barcode)
                barcode_dict[start_pos]=barcode_list
        else:
            barcode_dict[start_pos]=[barcode]
        if counter % 100000 == 0:
            print("Processed ", counter, "reads")
        
    return barcode_dict, counter, dup_count


In [32]:
data    = tn.read_samfile("sorted_reads/mapped_A1_R1_001.sam")
reads   = data[1][0:100000]
fwd_strand_reads = tn.filter_reads_by_strand(reads)[0]
results = make_barcode_dict(fwd_strand_reads)
barcode_dict = results[0]
print("Number of reads: ", len(fwd_strand_reads))
print("number of reads processed: ", results[1])
print("number of duplicate reads: ", results[2])
print("number of unique reads (len of dict): ", len(barcode_dict))
print("number of unique reads: ", results[1]-results[2])

KeyboardInterrupt: 

Reading entire samfile into memory was taking too long. Process one read at a time instead of extracting reads and sorting by strand.

1. like with fastq--enumerate and take every other line? Each line after header is a read.

In [54]:
def remove_dup_reads(sam_file):
    """ 
    Function to remove duplicate reads

    Input               sam_file                            .sam file of mapped reads
                        
    Output              unique_list_f                       list of unique forward reads
                        unique_list_r                       list of unique reverse reads
                                                        
    """
    import scripts.tnseq_pro as tn
    import numpy as np

    counter = 0
    barcode_list_f = []
    barcode_list_r = []

    #read sam_file and sort lines between header and reads too big for memory
    with open(sam_file, 'r') as f:
        for line in f:
            line = line.strip()
            if line.startswith("@"):
                pass
            else:
                start_pos = line.split()[3]
                counter += 1
                if counter % 1000000 == 0:
                    print("Processed ", counter, "reads")
                #find barcode and strand
                read_name  = line.split()[0]
                strand     = tn.parse_samflag(line)
                barcode    = read_name.split("BC:",1)[1]

                #add to dictionary of starts and barcodes 
                if strand == "F":
                # if start_pos in barcode_dict_f:
                #     barcode_list = barcode_dict_f[start_pos]
                #     if barcode in barcode_list:
                #         pass
                #         dup_count += 1
                #     else:
                #         barcode_list.append(barcode)
                #         barcode_dict_f[start_pos]=barcode_list
                # else:
                #     barcode_dict_f[start_pos]=[barcode]
                    barcode_list_f.append([start_pos, barcode])
                else:
                # if start_pos in barcode_dict_r:
                #     barcode_list = barcode_dict_r[start_pos]
                #     if barcode in barcode_list:
                #         pass
                #         dup_count += 1
                #     else:
                #         barcode_list.append(barcode)
                #         barcode_dict_r[start_pos]=barcode_list
                # else:
                #     barcode_dict_r[start_pos]=[barcode]
                    barcode_list_r.append([start_pos, barcode])

            #

    unique_list_f = list(np.unique(np.array(barcode_list_f), axis=0))
    unique_list_r = list(np.unique(np.array(barcode_list_r), axis=0))

    print("Total number of reads processed: ", counter)
    print("Total number of unique forward reads: ", len(unique_list_f))
    print("Total number of unique reverse reads: ", len(unique_list_r))
    print("Total number of duplicate reads: ", counter - len(unique_list_f) - len(unique_list_r))

    return unique_list_f, unique_list_r



In [3]:
mySam    = "sorted_reads/mapped_A1_R1_001.sam"


results = remove_dup_reads(mySam)


NameError: name 'remove_dup_reads' is not defined

Takes about 2 min for first 100000 reads. This will take about 22 hours at this rate. Will it be much faster on server? Maybe calculating modulo each read is also slowing it down. tried without counter report--no improvement. 

# on thoth: iterate barcoding script

In [8]:
import glob

import tnseq_pro as tn
#add barcode to fastq files on server
my_dir = "/d/in16/u/sj003/men_tnseq/"
#list files for file1
read1_files = glob.glob("/d/in16/u/sj003/men_tnseq/fastq/*_R1_001.fastq")
for read1_file in read1_files:
    index_file = read1_file.replace("R1", "R2")
    print(read1_file, index_file)
    outdir = my_dir + "barcoded_reads/"
    tn.add_barcode(read1_file, index_file, outdir)



/d/in16/u/sj003/men_tnseq/fastq/A1_R1_001.fastq /d/in16/u/sj003/men_tnseq/fastq/A1_R2_001.fastq
/d/in16/u/sj003/men_tnseq/fastq/A2_R1_001.fastq /d/in16/u/sj003/men_tnseq/fastq/A2_R2_001.fastq
/d/in16/u/sj003/men_tnseq/fastq/B1_R1_001.fastq /d/in16/u/sj003/men_tnseq/fastq/B1_R2_001.fastq
/d/in16/u/sj003/men_tnseq/fastq/B2_R1_001.fastq /d/in16/u/sj003/men_tnseq/fastq/B2_R2_001.fastq
/d/in16/u/sj003/men_tnseq/fastq/C1_R1_001.fastq /d/in16/u/sj003/men_tnseq/fastq/C1_R2_001.fastq
/d/in16/u/sj003/men_tnseq/fastq/C2_R1_001.fastq /d/in16/u/sj003/men_tnseq/fastq/C2_R2_001.fastq
/d/in16/u/sj003/men_tnseq/fastq/D1_R1_001.fastq /d/in16/u/sj003/men_tnseq/fastq/D1_R2_001.fastq
/d/in16/u/sj003/men_tnseq/fastq/D2_R1_001.fastq /d/in16/u/sj003/men_tnseq/fastq/D2_R2_001.fastq
/d/in16/u/sj003/men_tnseq/fastq/Mb09_R1_001.fastq /d/in16/u/sj003/men_tnseq/fastq/Mb09_R2_001.fastq
/d/in16/u/sj003/men_tnseq/fastq/Mb24_R1_001.fastq /d/in16/u/sj003/men_tnseq/fastq/Mb24_R2_001.fastq


## fastp to trim any remaining adapters

Fastp needs to work on fastq files--not on the barcoded files which are reduced to header/sequence. Re-do with fastp first and barcoding second.

In [ ]:
!conda activate snakemake
!nohup snakemake --cores 8 -s $my_path/snakemake/tnseq/fastp/snakefile.smk > nohup_fastp.out 2>&1 &


## Barcode each trimmed file with iterate script

In [1]:
import tnseq_pro as tn
trimmed_reads_dir = "fastq/trimmed"
barcoded_dir = "barcoded_reads"
tn.iterate_add_barcode(trimmed_reads_dir, barcoded_dir)

['fastq/trimmed/trimmed_B2_R1_001.fastq', 'fastq/trimmed/trimmed_D2_R1_001.fastq']
barcoded_reads/barcode_trimmed_B2_R1_001.fastq
barcoded_reads/barcode_trimmed_D2_R1_001.fastq


In [2]:
#check number of reads (should be 1/2 as many lines since 2 lines per read)
!for file in barcoded_reads/*.fastq; do
!    echo $(cat ${file} | wc -l)/2 | bc
!done

for: Command not found.
do: Command not found.
Illegal variable name.
done: Command not found.


## Trim transposon tag from adaptor-trimmed, barcoded reads

In [1]:
# use trim_tags_fastq(fastq_file, outdir, tag, mismatch_max)
import tnseq_pro as tn
import glob
barcoded_files = glob.glob("barcoded_reads" + "/*.fastq")
for file in barcoded_files:
    print("File being processed: ", file)
    tn.trim_tag_fastq(file, "barcoded_reads/tag_clipped/")

File being processed:  barcoded_reads/barcode_trimmed_B2_R1_001.fastq
Processed  5000000 reads
Processed  10000000 reads
Processed  15000000 reads
Processed  20000000 reads
Processed  25000000 reads
Processed  30000000 reads
Processed  35000000 reads
Processed  40000000 reads
Processed  45000000 reads
Processed  50000000 reads
Processed  55000000 reads
Total number of reads processed:  59567959
Total number of reads with tag:  58555393.0
Total number of reads without tag:  1012566.0
File being processed:  barcoded_reads/barcode_trimmed_D2_R1_001.fastq
Processed  5000000 reads
Total number of reads processed:  5051974
Total number of reads with tag:  4917158.0
Total number of reads without tag:  134816.0
File being processed:  barcoded_reads/tag_clipped_barcode_trimmed_B2_R1_001.fastq
Total number of reads processed:  265243
Total number of reads with tag:  3.0
Total number of reads without tag:  265240.0
File being processed:  barcoded_reads/no_tag_barcode_trimmed_B2_R1_001.fastq
Proce

KeyboardInterrupt: 

TATCTAGTCTAGAGACCGGGGACTTATCAGCCAACCTGTTA is longest primer and therefore needs ~42 for match

Once that is fixed, no transposon reads are 2.6% for d2 and 1.7% for B2

In [ ]:
#check that tag-clipped files have correct number of reads, may be a bit less than number of barcoded reads
!for file in barcoded_reads/tag_clipped/*.fastq; do
!    echo ${file}
!    echo $(cat ${file} | wc -l)/2 | bc
!done

#how many no-tags
!for file in barcoded_reads/no_tags/*.fastq; do
!    echo ${file}
!    echo $(cat ${file} | wc -l)/2 | bc
!done



## map tag-clipped reads with bwa-mem

In [ ]:
!conda activate snakemake
!snakemake -np -s $my_path/snakemake/tnseq/mapping/snakefile.smk
!nohup snakemake --cores 8 -s $my_path/snakemake/tnseq/mapping/snakefile.smk > nohup.out 2>&1 &


### QC of mapped reads

In [2]:
import glob
flagstat_files = glob.glob("sorted_reads/*.txt")
for file in flagstat_files:
    print(file)
    with open(file, 'r') as f:
        for line in f:
            print(line.strip())
    print("\n")

sorted_reads/flagstat_D2_R1_001.txt
4918902 + 0 in total (QC-passed reads + QC-failed reads)
4917158 + 0 primary
0 + 0 secondary
1744 + 0 supplementary
0 + 0 duplicates
0 + 0 primary duplicates
4276826 + 0 mapped (86.95% : N/A)
4275082 + 0 primary mapped (86.94% : N/A)
0 + 0 paired in sequencing
0 + 0 read1
0 + 0 read2
0 + 0 properly paired (N/A : N/A)
0 + 0 with itself and mate mapped
0 + 0 singletons (N/A : N/A)
0 + 0 with mate mapped to a different chr
0 + 0 with mate mapped to a different chr (mapQ>=5)


sorted_reads/flagstat_C1_R1_001.txt
5041075 + 0 in total (QC-passed reads + QC-failed reads)
5040493 + 0 primary
0 + 0 secondary
582 + 0 supplementary
0 + 0 duplicates
0 + 0 primary duplicates
4170196 + 0 mapped (82.72% : N/A)
4169614 + 0 primary mapped (82.72% : N/A)
0 + 0 paired in sequencing
0 + 0 read1
0 + 0 read2
0 + 0 properly paired (N/A : N/A)
0 + 0 with itself and mate mapped
0 + 0 singletons (N/A : N/A)
0 + 0 with mate mapped to a different chr
0 + 0 with mate mapped to a

samples B2 and D2 are worst with < 1M reads D2 (~18471 reads mapped) maybe to be expected as we weren't able to amplify the junctions in this sample. B2 was close to 1 nM concentration according to qPCR analysis.

checked length of barcoded/tag clipped files and both were severely reduced here. will redo for these two files--perhaps were incomplete.

Re-done and it appears moved to no tag but they really have tags still on them. Figured out it was the primer being too long and exceeding the search space on the sequence allowed to find the tag. extended this and got better results. There are more tags there, but a deletion or insertion early on will make more than 2 mismatches. some of the reads I saw with more mismatches also had very short reads. For B2 really doesn't matter as 59M reads so 1M lost isn't such a big deal. For D2 this is 2.6% of reads. Could see if more lenient mismatch helps rescue some reads for this sample.

re-mapped B2 and D2 and have >4M unique reads mapped for D2 and >54M unique reads mapped for B2.

## Quantification and filtering for duplicate reads

In [ ]:
import tnseq_pro as tn
tn.iterate_sam_to_wig("sorted_reads", "/d/in16/u/sj003/refseqs/mbovis/Mbovis_AF2122_97.fasta")

['sorted_reads/mapped_C1_R1_001.sam', 'sorted_reads/mapped_Mb24_R1_001.sam', 'sorted_reads/mapped_C2_R1_001.sam', 'sorted_reads/mapped_B1_R1_001.sam', 'sorted_reads/mapped_Mb09_R1_001.sam', 'sorted_reads/mapped_B2_R1_001.sam', 'sorted_reads/mapped_A2_R1_001.sam', 'sorted_reads/mapped_D2_R1_001.sam', 'sorted_reads/mapped_A1_R1_001.sam', 'sorted_reads/mapped_D1_R1_001.sam']
C1
sorted_reads/mapped_C1_R1_001.sam
Total number of reads processed:  4170196
Total number of unique forward reads:  2027715
Total number of unique reverse reads:  2011725
Total number of duplicate reads:  130756
number of unique reads assigned to TA sites:  4000616
number of unique reads with no ta site match:  38824
Mb24
sorted_reads/mapped_Mb24_R1_001.sam
Processed  5000000 reads
Processed  10000000 reads
Processed  15000000 reads
Processed  20000000 reads
Total number of reads processed:  20073840
Total number of unique forward reads:  7631531
Total number of unique reverse reads:  7468303
Total number of duplica

In [2]:
import tnseq_pro as tn
tn.sam_to_wig("sorted_reads/mapped_D1_R1_001.sam","/d/in16/u/sj003/refseqs/mbovis/Mbovis_AF2122_97.fasta", "D1")

Processed  5000000 reads
Processed  10000000 reads
Processed  15000000 reads
Processed  20000000 reads
Processed  25000000 reads
Processed  30000000 reads
Processed  35000000 reads
Processed  40000000 reads
Total number of reads processed:  44185634
Total number of unique forward reads:  20354806
Total number of unique reverse reads:  20163248
Total number of duplicate reads:  3667580
number of unique reads assigned to TA sites:  40321820
number of unique reads with no ta site match:  196234


'output/D1_insertions.wig'